# C 240p

In [1]:
%%writefile C_240p.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define NUM_FRAMES 8
#define INPUT_TEMPLATE_PATH "./data/template_240.jpg"
#define INPUT_FRAME_PATH_FORMAT "./data/images240p/%d.jpg"
#define OUTPUT_FRAME_PATH_FORMAT "./results/240p/frame_%02d.jpg"

void color_to_grayscale(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels);
void find_best_match(const unsigned char *frame_gray, int frame_w, int frame_h, const unsigned char *template_gray, int template_w, int template_h, double template_norm, int *best_index);
void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *frame_gray, int frame_w, unsigned char *template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {
    //variables for timer
    clock_t start, end;
    double elapse, time_taken;
    int timed_calls = 0; 

    printf("Starting 240p C object tracker...\n");

    int template_w, template_h, template_c;

    unsigned char *template_color = stbi_load(INPUT_TEMPLATE_PATH, &template_w, &template_h, &template_c, 0);
    if (template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load initial template image from %s\n", INPUT_TEMPLATE_PATH);
        return 1;
    }

    unsigned char *template_gray = (unsigned char *)malloc(template_w * template_h * sizeof(unsigned char));
    if (template_gray == NULL) {
        fprintf(stderr, "ERROR: Could not allocate memory for grayscale template.\n");
        stbi_image_free(template_color);
        return 1;
    }

    //add to the total kernel time the time it takes to convert the template image from color to gray
    start = clock();
    color_to_grayscale(template_color, template_gray, template_w, template_h, template_c);
    end = clock();

    //TODO comment mo this
    //printf("template_w = %d, template_h = %d, template_c = %d\n",  template_w, template_h, template_c );

    time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC; // milliseconds
    elapse += time_taken;
    timed_calls++;

    stbi_image_free(template_color); 

    //compute the norm of the template image
    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) {
        template_norm_sq += (double)template_gray[i] * template_gray[i];
    }
    double template_norm = sqrt(template_norm_sq);

    //printf("first template norm: %lf\n", template_norm);

    for (int i = 0; i < NUM_FRAMES; ++i) {
        
        char frame_path[128];
        sprintf(frame_path, INPUT_FRAME_PATH_FORMAT, i);

        int frame_w, frame_h, frame_c;
        unsigned char *frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 0);
        if (frame_color == NULL) {
            fprintf(stderr, "Warning: Could not load frame %s. Skipping.\n", frame_path);
            continue;
        }

        unsigned char *frame_gray = (unsigned char *)malloc(frame_w * frame_h * sizeof(unsigned char));
        if (frame_gray == NULL) {
            fprintf(stderr, "ERROR: Could not allocate memory for grayscale frame %d.\n", i);
            stbi_image_free(frame_color);
            continue;
        }
        

        int best_x, best_y;
        int best_index = 0;
        
        start = clock();
        color_to_grayscale(frame_color, frame_gray, frame_w, frame_h, frame_c);

        find_best_match(frame_gray, frame_w, frame_h, template_gray, template_w, template_h, template_norm, &best_index);
        end = clock();
        timed_calls++;

        //best_index is the top leftmost index of the best matching patch
        best_x = best_index % frame_w;
        best_y = best_index / frame_w;
        printf("Frame %d: Best match found at (x=%d, y=%d)\n", i, best_x, best_y);
        draw_rectangle(frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);
        
        char output_path[128];
        sprintf(output_path, OUTPUT_FRAME_PATH_FORMAT, i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, frame_color, frame_w * frame_c);

        update_template(frame_gray, frame_w, template_gray, template_w, template_h, best_x, best_y);
        
        //recalculate template_norm for the updated template
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)template_gray[j] * template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);
        //printf("template norm[i]: %lf\n", template_norm);
        
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
        

        stbi_image_free(frame_color);
        free(frame_gray);
    }

    
    printf("Kernel Execution Time: %f milliseconds\n", elapse);
    printf("Number of Recorded Kernel Calls: %d\n",timed_calls);

    free(template_gray);
    printf("Image results saved in 'results/240p' directory.\n");

    return 0;
}


void color_to_grayscale(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    //loop through each row of the image
    for (int y = 0; y < height; ++y) {
        //loop through each pixel(column)
        for (int x = 0; x < width; ++x) {
            //color_idx is the index of the red component of the current pixel
            int color_idx = (y * width + x) * channels;
            unsigned char r = color_img[color_idx];
            unsigned char g = color_img[color_idx + 1];
            unsigned char b = color_img[color_idx + 2];
            
            double gray_val = 0.299f * r + 0.587f * g + 0.114f * b;

            //unsigned char ranges from 0-255
            gray_img[y * width + x] = (unsigned char)gray_val;
        }
    }
}


void find_best_match(const unsigned char *frame_gray, int frame_w, int frame_h, const unsigned char *template_gray, int template_w, int template_h, double template_norm, int *best_index) {
    double max_similarity = -2.0;

    //TODO: comment these
    //printf("frame_h - template_h = %d\n", frame_h - template_h);
    //printf("frame_w - template_w = %d\n", frame_w - template_w);

    // so for 240p(426x240), the search space for row in the current frame image is only 0 to 122; 122+118=240 which is index 239
    for (int y = 0; y <= frame_h - template_h; y++) {
        for (int x = 0; x <= frame_w - template_w; x++) {
            double dot_product = 0.0;
            double patch_norm_sq = 0.0;


            for (int ty = 0; ty < template_h; ++ty) {
                for (int tx = 0; tx < template_w; ++tx) {
                    int frame_idx = (y + ty) * frame_w + (x + tx);
                    int template_idx = ty * template_w + tx;
                    
                    dot_product += (double)frame_gray[frame_idx] * template_gray[template_idx];
                    patch_norm_sq += (double)frame_gray[frame_idx] * frame_gray[frame_idx];
                }
            }
            double patch_norm = sqrt(patch_norm_sq);

            double similarity;
            if (patch_norm == 0 || template_norm == 0) {
                similarity = 0.0f;
            } else {
                similarity = (double)(dot_product / (patch_norm * template_norm));
            }

            if (similarity > max_similarity) {
                max_similarity = similarity;
                *best_index = y * frame_w + x;
            }
        }
    }
}



void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) {
                memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            }
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) {
                memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
            }
        }
    }

    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) {
                memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            }
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) {
                memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
            }
        }
    }
}

void update_template(const unsigned char *frame_gray, int frame_w, unsigned char *template_gray, int template_w, int template_h, int best_x, int best_y) {
    for (int ty = 0; ty < template_h; ++ty) {
        for (int tx = 0; tx < template_w; ++tx) {
            int frame_idx = (best_y + ty) * frame_w + (best_x + tx);
            int template_idx = ty * template_w + tx;
            template_gray[template_idx] = frame_gray[frame_idx];
        }
    }
}

Writing C_240p.c


In [2]:
%%bash
gcc C_240p.c -o C_240p -lm

In [3]:
%%bash
./C_240p

Starting 240p C object tracker...
Frame 0: Best match found at (x=138, y=46)
Frame 1: Best match found at (x=147, y=49)
Frame 2: Best match found at (x=160, y=47)
Frame 3: Best match found at (x=180, y=52)
Frame 4: Best match found at (x=170, y=51)
Frame 5: Best match found at (x=151, y=47)
Frame 6: Best match found at (x=129, y=50)
Frame 7: Best match found at (x=125, y=51)
Kernel Execution Time: 26710.191000 milliseconds
Number of Recorded Kernel Calls: 9
Image results saved in 'results/240p' directory.


# C 480p

In [1]:
%%writefile C_480p.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define NUM_FRAMES 8
#define INPUT_TEMPLATE_PATH "./data/template_480.jpg"
#define INPUT_FRAME_PATH_FORMAT "./data/images480p/%d.jpg"
#define OUTPUT_FRAME_PATH_FORMAT "./results/480p/frame_%02d.jpg"

void color_to_grayscale(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels);
void find_best_match(const unsigned char *frame_gray, int frame_w, int frame_h, const unsigned char *template_gray, int template_w, int template_h, double template_norm, int *best_index);
void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *frame_gray, int frame_w, unsigned char *template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {
    //variables for timer
    clock_t start, end;
    double elapse, time_taken;
    int timed_calls = 0; 

    printf("Starting 480p C object tracker...\n");

    int template_w, template_h, template_c;

    unsigned char *template_color = stbi_load(INPUT_TEMPLATE_PATH, &template_w, &template_h, &template_c, 0);
    if (template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load initial template image from %s\n", INPUT_TEMPLATE_PATH);
        return 1;
    }

    unsigned char *template_gray = (unsigned char *)malloc(template_w * template_h * sizeof(unsigned char));
    if (template_gray == NULL) {
        fprintf(stderr, "ERROR: Could not allocate memory for grayscale template.\n");
        stbi_image_free(template_color);
        return 1;
    }

    //add to the total kernel time the time it takes to convert the template image from color to gray
    start = clock();
    color_to_grayscale(template_color, template_gray, template_w, template_h, template_c);
    end = clock();

    //TODO comment mo this
    //printf("template_w = %d, template_h = %d, template_c = %d\n",  template_w, template_h, template_c );

    time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC; // milliseconds
    elapse += time_taken;
    timed_calls++;

    
    stbi_image_free(template_color); 

    //compute the norm of the template image
    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) {
        template_norm_sq += (double)template_gray[i] * template_gray[i];
    }
    double template_norm = sqrt(template_norm_sq);


    for (int i = 0; i < NUM_FRAMES; ++i) {
        
        char frame_path[128];
        sprintf(frame_path, INPUT_FRAME_PATH_FORMAT, i);

        int frame_w, frame_h, frame_c;
        unsigned char *frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 0);
        if (frame_color == NULL) {
            fprintf(stderr, "Warning: Could not load frame %s. Skipping.\n", frame_path);
            continue;
        }

        unsigned char *frame_gray = (unsigned char *)malloc(frame_w * frame_h * sizeof(unsigned char));
        if (frame_gray == NULL) {
            fprintf(stderr, "ERROR: Could not allocate memory for grayscale frame %d.\n", i);
            stbi_image_free(frame_color);
            continue;
        }
        

        int best_x, best_y;
        int best_index = 0;
        
        start = clock();
        color_to_grayscale(frame_color, frame_gray, frame_w, frame_h, frame_c);

        //TODO comment mo this
        //printf("frame_w = %d, frame_h = %d,frame_c = %d\n", frame_w, frame_h, frame_c);

        find_best_match(frame_gray, frame_w, frame_h, template_gray, template_w, template_h, template_norm, &best_index);
        end = clock();
        timed_calls++;

        //best_index is the top leftmost index of the best matching patch
        best_x = best_index % frame_w;
        best_y = best_index / frame_w;
        printf("Frame %d: Best match found at (x=%d, y=%d)\n", i, best_x, best_y);
        draw_rectangle(frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);
        
        char output_path[128];
        sprintf(output_path, OUTPUT_FRAME_PATH_FORMAT, i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, frame_color, frame_w * frame_c);

        update_template(frame_gray, frame_w, template_gray, template_w, template_h, best_x, best_y);
        
        //recalculate template_norm for the updated template
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)template_gray[j] * template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);
        //printf("template norm[i]: %lf\n", template_norm);
        
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
        

        stbi_image_free(frame_color);
        free(frame_gray);
    }

    
    printf("Kernel Execution Time: %f milliseconds\n", elapse);
    printf("Number of Recorded Kernel Calls: %d\n",timed_calls);

    free(template_gray);
    printf("Image results saved in 'results/240p' directory.\n");

    return 0;
}


void color_to_grayscale(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    //loop through each row of the image
    for (int y = 0; y < height; ++y) {
        //loop through each pixel(column)
        for (int x = 0; x < width; ++x) {
            //color_idx is the index of the red component of the current pixel
            int color_idx = (y * width + x) * channels;
            unsigned char r = color_img[color_idx];
            unsigned char g = color_img[color_idx + 1];
            unsigned char b = color_img[color_idx + 2];
            
            double gray_val = 0.299f * r + 0.587f * g + 0.114f * b;

            //unsigned char ranges from 0-255
            gray_img[y * width + x] = (unsigned char)gray_val;
        }
    }
}


void find_best_match(const unsigned char *frame_gray, int frame_w, int frame_h, const unsigned char *template_gray, int template_w, int template_h, double template_norm, int *best_index) {
    double max_similarity = -2.0;

    //TODO: comment these
    //printf("frame_h - template_h = %d\n", frame_h - template_h);
    //printf("frame_w - template_w = %d\n", frame_w - template_w);

    for (int y = 0; y <= frame_h - template_h; y++) {
        for (int x = 0; x <= frame_w - template_w; x++) {
            double dot_product = 0.0;
            double patch_norm_sq = 0.0;


            for (int ty = 0; ty < template_h; ++ty) {
                for (int tx = 0; tx < template_w; ++tx) {
                    int frame_idx = (y + ty) * frame_w + (x + tx);
                    int template_idx = ty * template_w + tx;
                    
                    dot_product += (double)frame_gray[frame_idx] * template_gray[template_idx];
                    patch_norm_sq += (double)frame_gray[frame_idx] * frame_gray[frame_idx];
                }
            }
            double patch_norm = sqrt(patch_norm_sq);

            double similarity;
            if (patch_norm == 0 || template_norm == 0) {
                similarity = 0.0f;
            } else {
                similarity = (double)(dot_product / (patch_norm * template_norm));
            }

            if (similarity > max_similarity) {
                max_similarity = similarity;
                *best_index = y * frame_w + x;
            }
        }
    }
}



void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) {
                memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            }
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) {
                memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
            }
        }
    }

    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) {
                memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            }
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) {
                memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
            }
        }
    }
}

void update_template(const unsigned char *frame_gray, int frame_w, unsigned char *template_gray, int template_w, int template_h, int best_x, int best_y) {
    for (int ty = 0; ty < template_h; ++ty) {
        for (int tx = 0; tx < template_w; ++tx) {
            int frame_idx = (best_y + ty) * frame_w + (best_x + tx);
            int template_idx = ty * template_w + tx;
            template_gray[template_idx] = frame_gray[frame_idx];
        }
    }
}

Overwriting C_480p.c


In [2]:
%%bash
gcc C_480p.c -o C_480p -lm

In [3]:
%%bash
./C_480p

Starting 480p C object tracker...
Frame 0: Best match found at (x=289, y=90)
Frame 1: Best match found at (x=300, y=106)
Frame 2: Best match found at (x=329, y=105)
Frame 3: Best match found at (x=372, y=114)
Frame 4: Best match found at (x=351, y=112)
Frame 5: Best match found at (x=312, y=103)
Frame 6: Best match found at (x=268, y=107)
Frame 7: Best match found at (x=261, y=110)
Kernel Execution Time: 494972.636000 milliseconds
Number of Recorded Kernel Calls: 9
Image results saved in 'results/240p' directory.


# C 720p

In [4]:
%%writefile C_720p.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define NUM_FRAMES 8
#define INPUT_TEMPLATE_PATH "./data/template_720.jpg"
#define INPUT_FRAME_PATH_FORMAT "./data/images720p/%d.jpg"
#define OUTPUT_FRAME_PATH_FORMAT "./results/720p/frame_%02d.jpg"

void color_to_grayscale(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels);
void find_best_match(const unsigned char *frame_gray, int frame_w, int frame_h, const unsigned char *template_gray, int template_w, int template_h, double template_norm, int *best_index);
void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *frame_gray, int frame_w, unsigned char *template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {
    //variables for timer
    clock_t start, end;
    double elapse, time_taken;
    int timed_calls = 0; 

    printf("Starting 720p C object tracker...\n");

    int template_w, template_h, template_c;

    unsigned char *template_color = stbi_load(INPUT_TEMPLATE_PATH, &template_w, &template_h, &template_c, 0);
    if (template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load initial template image from %s\n", INPUT_TEMPLATE_PATH);
        return 1;
    }

    unsigned char *template_gray = (unsigned char *)malloc(template_w * template_h * sizeof(unsigned char));
    if (template_gray == NULL) {
        fprintf(stderr, "ERROR: Could not allocate memory for grayscale template.\n");
        stbi_image_free(template_color);
        return 1;
    }

    //add to the total kernel time the time it takes to convert the template image from color to gray
    start = clock();
    color_to_grayscale(template_color, template_gray, template_w, template_h, template_c);
    end = clock();

    //TODO comment mo this
    //printf("template_w = %d, template_h = %d, template_c = %d\n",  template_w, template_h, template_c );

    time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC; // milliseconds
    elapse += time_taken;
    timed_calls++;

    
    stbi_image_free(template_color); 

    //compute the norm of the template image
    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) {
        template_norm_sq += (double)template_gray[i] * template_gray[i];
    }
    double template_norm = sqrt(template_norm_sq);

    for (int i = 0; i < NUM_FRAMES; ++i) {
        
        char frame_path[128];
        sprintf(frame_path, INPUT_FRAME_PATH_FORMAT, i);

        int frame_w, frame_h, frame_c;
        unsigned char *frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 0);
        if (frame_color == NULL) {
            fprintf(stderr, "Warning: Could not load frame %s. Skipping.\n", frame_path);
            continue;
        }

        unsigned char *frame_gray = (unsigned char *)malloc(frame_w * frame_h * sizeof(unsigned char));
        if (frame_gray == NULL) {
            fprintf(stderr, "ERROR: Could not allocate memory for grayscale frame %d.\n", i);
            stbi_image_free(frame_color);
            continue;
        }
        

        int best_x, best_y;
        int best_index = 0;
        
        start = clock();
        color_to_grayscale(frame_color, frame_gray, frame_w, frame_h, frame_c);

        //TODO comment mo this
        //printf("frame_w = %d, frame_h = %d,frame_c = %d\n", frame_w, frame_h, frame_c);

        find_best_match(frame_gray, frame_w, frame_h, template_gray, template_w, template_h, template_norm, &best_index);
        end = clock();
        timed_calls++;

        //best_index is the top leftmost index of the best matching patch
        best_x = best_index % frame_w;
        best_y = best_index / frame_w;
        printf("Frame %d: Best match found at (x=%d, y=%d)\n", i, best_x, best_y);
        draw_rectangle(frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);
        
        char output_path[128];
        sprintf(output_path, OUTPUT_FRAME_PATH_FORMAT, i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, frame_color, frame_w * frame_c);

        update_template(frame_gray, frame_w, template_gray, template_w, template_h, best_x, best_y);
        
        //recalculate template_norm for the updated template
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)template_gray[j] * template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);
        //printf("template norm[i]: %lf\n", template_norm);
        
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
        

        stbi_image_free(frame_color);
        free(frame_gray);
    }

    
    printf("Kernel Execution Time: %f milliseconds\n", elapse);
    printf("Number of Recorded Kernel Calls: %d\n",timed_calls);

    free(template_gray);
    printf("Image results saved in 'results/720p' directory.\n");

    return 0;
}


void color_to_grayscale(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    //loop through each row of the image
    for (int y = 0; y < height; ++y) {
        //loop through each pixel(column)
        for (int x = 0; x < width; ++x) {
            //color_idx is the index of the red component of the current pixel
            int color_idx = (y * width + x) * channels;
            unsigned char r = color_img[color_idx];
            unsigned char g = color_img[color_idx + 1];
            unsigned char b = color_img[color_idx + 2];
            
            double gray_val = 0.299f * r + 0.587f * g + 0.114f * b;

            //unsigned char ranges from 0-255
            gray_img[y * width + x] = (unsigned char)gray_val;
        }
    }
}


void find_best_match(const unsigned char *frame_gray, int frame_w, int frame_h, const unsigned char *template_gray, int template_w, int template_h, double template_norm, int *best_index) {
    double max_similarity = -2.0;

    //TODO: comment these
    //printf("frame_h - template_h = %d\n", frame_h - template_h);
    //printf("frame_w - template_w = %d\n", frame_w - template_w);

    for (int y = 0; y <= frame_h - template_h; y++) {
        for (int x = 0; x <= frame_w - template_w; x++) {
            double dot_product = 0.0;
            double patch_norm_sq = 0.0;


            for (int ty = 0; ty < template_h; ++ty) {
                for (int tx = 0; tx < template_w; ++tx) {
                    int frame_idx = (y + ty) * frame_w + (x + tx);
                    int template_idx = ty * template_w + tx;
                    
                    dot_product += (double)frame_gray[frame_idx] * template_gray[template_idx];
                    patch_norm_sq += (double)frame_gray[frame_idx] * frame_gray[frame_idx];
                }
            }
            double patch_norm = sqrt(patch_norm_sq);

            double similarity;
            if (patch_norm == 0 || template_norm == 0) {
                similarity = 0.0f;
            } else {
                similarity = (double)(dot_product / (patch_norm * template_norm));
            }

            if (similarity > max_similarity) {
                max_similarity = similarity;
                *best_index = y * frame_w + x;
            }
        }
    }
}



void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) {
                memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            }
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) {
                memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
            }
        }
    }

    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) {
                memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            }
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) {
                memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
            }
        }
    }
}

void update_template(const unsigned char *frame_gray, int frame_w, unsigned char *template_gray, int template_w, int template_h, int best_x, int best_y) {
    for (int ty = 0; ty < template_h; ++ty) {
        for (int tx = 0; tx < template_w; ++tx) {
            int frame_idx = (best_y + ty) * frame_w + (best_x + tx);
            int template_idx = ty * template_w + tx;
            template_gray[template_idx] = frame_gray[frame_idx];
        }
    }
}

Overwriting C_720p.c


In [5]:
%%bash
gcc C_720p.c -o C_720p -lm

In [6]:
%%bash
./C_720p

Starting 720p C object tracker...
Frame 0: Best match found at (x=426, y=131)
Frame 1: Best match found at (x=453, y=141)
Frame 2: Best match found at (x=493, y=132)
Frame 3: Best match found at (x=556, y=148)
Frame 4: Best match found at (x=526, y=144)
Frame 5: Best match found at (x=468, y=131)
Frame 6: Best match found at (x=401, y=139)
Frame 7: Best match found at (x=389, y=143)
Kernel Execution Time: 2691901.432000 milliseconds
Number of Recorded Kernel Calls: 9
Image results saved in 'results/720p' directory.


# Add CUDA to path

In [4]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


# CUDA 240p Shared Memory

In [31]:
%%writefile CUDA_240p.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

//this function is just for error checking
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define BLOCK_SIZE 16
#define REDUCTION_THREADS 256

__global__ 
void color_to_grayscale_kernel(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        //index for the gray_img
        int gray_idx = y * width + x;
        //index for color_img
        int color_idx = gray_idx * channels; 

        unsigned char r = color_img[color_idx];
        unsigned char g = color_img[color_idx + 1];
        unsigned char b = color_img[color_idx + 2];

        gray_img[gray_idx] = (unsigned char)(0.299f * r + 0.587f * g + 0.114f * b);
    }
}

__global__ 
void find_best_match_kernel(const unsigned char *frame_gray, int frame_w, int frame_h,
                            const unsigned char *template_gray, int template_w, int template_h,
                            double template_norm, double *similarity_map) {
    
    extern __shared__ unsigned char shared_template[];

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    int tid = threadIdx.y * blockDim.x + threadIdx.x;
    int block_threads = blockDim.x * blockDim.y;

    int template_size = template_w * template_h;
    for (int i = tid; i < template_size; i += block_threads) {
        shared_template[i] = template_gray[i];
    }

    __syncthreads();

    if (x <= frame_w - template_w && y <= frame_h - template_h) {
        double dot_product = 0.0;
        double patch_norm_sq = 0.0;

        for (int ty = 0; ty < template_h; ty++) {
            for (int tx = 0; tx < template_w; tx++) {
                int frame_idx = (y + ty) * frame_w + (x + tx);
                int template_idx = ty * template_w + tx;
                
                unsigned char frame_val = frame_gray[frame_idx];
                unsigned char template_val = shared_template[template_idx];
                
                dot_product += (double)frame_val * template_val;
                patch_norm_sq += (double)frame_val * frame_val;
            }
        }

        double patch_norm = sqrt(patch_norm_sq);
        double similarity;
        if (patch_norm == 0 || template_norm == 0) {
            similarity = 0.0f;
        } else {
            similarity = (double)(dot_product / (patch_norm * template_norm));
        }

        similarity_map[y * (frame_w - template_w + 1) + x] = similarity;
    }
}
__global__
void find_max_kernel(const double *similarity_map, int map_size, 
                     double *block_max_vals, int *block_max_idxs) {
    __shared__ double shared_vals[REDUCTION_THREADS];
    __shared__ int shared_idxs[REDUCTION_THREADS];

    int tid = threadIdx.x; //Thread's position within block 
    int idx = blockIdx.x * blockDim.x + threadIdx.x; //Index in the similarity map where the thread will get the value

    double my_val = -2.0;
    int my_idx = 0;
    
    if (idx < map_size) {
        my_val = similarity_map[idx]; //get the id's value in the similarity map
        my_idx = idx;
    }

    shared_vals[tid] = my_val; // write value in the shared memory; shared memory is shared within the block
    shared_idxs[tid] = my_idx; // store the global position across all blocks 
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s/=2) {
        if (tid < s) {
            if (shared_vals[tid + s] > shared_vals[tid]) { //example: compare value at index 0 to value at index 128, then store which is higher
                shared_vals[tid] = shared_vals[tid + s];
                shared_idxs[tid] = shared_idxs[tid + s];
            }
        }
        __syncthreads();
    }
    if (tid == 0) {
        block_max_vals[blockIdx.x] = shared_vals[0]; // best is at index 0
        block_max_idxs[blockIdx.x] = shared_idxs[0];
    }
}

void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {
    printf("Starting CUDA 240p object tracker (WITH SHARED TEMPLATE VERSION)\n");

    int template_w, template_h, template_c;
    unsigned char *h_template_color = stbi_load("data/template_240.jpg", &template_w, &template_h, &template_c, 3);
    if (h_template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load template image 'data/template_240.jpg'. Check the file path.\n");
        return 1;
    }
    template_c = 3; 
    printf("Template loaded successfully (%d x %d).\n", template_w, template_h);

    unsigned char *d_template_color, *d_template_gray;

    // allocate space in the GPU for the template color and grayscale
    gpuErrchk( cudaMalloc((void**)&d_template_color, template_w * template_h * 3) );
    gpuErrchk( cudaMalloc((void**)&d_template_gray, template_w * template_h) );

    //Memcpy(dst,src)
    gpuErrchk( cudaMemcpy(d_template_color, h_template_color, template_w * template_h * 3, cudaMemcpyHostToDevice) );
    
    dim3 grid_template((template_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (template_h + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block_template(BLOCK_SIZE, BLOCK_SIZE);
    color_to_grayscale_kernel<<<grid_template, block_template>>>(d_template_color, d_template_gray, template_w, template_h, 3);
    gpuErrchk( cudaPeekAtLastError() );
    
    gpuErrchk( cudaFree(d_template_color) );
    stbi_image_free(h_template_color);

    // Calculate template norm on CPU
    unsigned char *h_template_gray = (unsigned char *)malloc(template_w * template_h);
    gpuErrchk( cudaMemcpy(h_template_gray, d_template_gray, template_w * template_h, cudaMemcpyDeviceToHost) );

    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) template_norm_sq += (double)h_template_gray[i] * h_template_gray[i];
    double template_norm = sqrt(template_norm_sq);

    for (int i = 0; i < 8; ++i) {
        char frame_path[128];
        sprintf(frame_path, "data/images240p/%d.jpg", i);

        int frame_w, frame_h, frame_c;
        unsigned char *h_frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 3);
        if (h_frame_color == NULL) {
            fprintf(stderr, "ERROR: Could not load frame %d from '%s'. Skipping.\n", i, frame_path);
            continue;
        }
        frame_c = 3;

        unsigned char *d_frame_color, *d_frame_gray;
        double *d_similarity_map; //for storing the similarity scores
        int map_w = frame_w - template_w + 1;
        int map_h = frame_h - template_h + 1;

        //allocate to global memory
        gpuErrchk( cudaMalloc((void**)&d_frame_color, frame_w * frame_h * frame_c) );
        gpuErrchk( cudaMalloc((void**)&d_frame_gray, frame_w * frame_h) );
        gpuErrchk( cudaMalloc((void**)&d_similarity_map, map_w * map_h * sizeof(double)) );
        gpuErrchk( cudaMemcpy(d_frame_color, h_frame_color, frame_w * frame_h * frame_c, cudaMemcpyHostToDevice) );


        dim3 grid_dim_gray( (frame_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (frame_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_gray(BLOCK_SIZE, BLOCK_SIZE);
        color_to_grayscale_kernel<<<grid_dim_gray, block_dim_gray>>>(d_frame_color, d_frame_gray, frame_w, frame_h, frame_c);
        gpuErrchk( cudaPeekAtLastError() );

        //smaller dimension since we process only valid template positions
        dim3 grid_dim_match( (map_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (map_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_match(BLOCK_SIZE, BLOCK_SIZE);

        int shared_mem_size = template_w * template_h * sizeof(unsigned char);
        
        find_best_match_kernel<<<grid_dim_match, block_dim_match, shared_mem_size>>>(d_frame_gray, frame_w, frame_h, d_template_gray, template_w, template_h, template_norm, d_similarity_map);
        gpuErrchk( cudaPeekAtLastError() );

        // Two-pass reduction
        int map_size = map_w * map_h;
        int threads = REDUCTION_THREADS;
        int blocks = (map_size + threads - 1) / threads;

        double *d_block_max;
        int *d_block_idx;
        gpuErrchk( cudaMalloc((void**)&d_block_max, blocks * sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_block_idx, blocks * sizeof(int)) );

        // First pass: get the best loc within each block
        find_max_kernel<<<blocks, threads>>>(d_similarity_map, map_size, d_block_max, d_block_idx);
        gpuErrchk( cudaPeekAtLastError() );

        // Second pass: get the final best loc among all the blocks 
        int best_x, best_y;
        double max_similarity;
        int global_idx;
        
        double *d_final_max;
        int *d_final_idx;
        gpuErrchk( cudaMalloc((void**)&d_final_max, sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_final_idx, sizeof(int)) );

        find_max_kernel<<<1, threads>>>(d_block_max, blocks, d_final_max, d_final_idx);//send 1 block of 256 threads because we are only processing ~164 blocks
        gpuErrchk( cudaPeekAtLastError() );

        int winning_block;//the block that holds the best location
        gpuErrchk( cudaMemcpy(&max_similarity, d_final_max, sizeof(double), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&winning_block, d_final_idx, sizeof(int), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&global_idx, &d_block_idx[winning_block], sizeof(int), cudaMemcpyDeviceToHost) );

        // same sa C
        best_x = global_idx % map_w;
        best_y = global_idx / map_w;

        gpuErrchk( cudaFree(d_block_max) );
        gpuErrchk( cudaFree(d_block_idx) );
        gpuErrchk( cudaFree(d_final_max) );
        gpuErrchk( cudaFree(d_final_idx) );

        printf("Frame %d: Best match found at (x=%d, y=%d) with similarity: %f\n", i, best_x, best_y, max_similarity);

        draw_rectangle(h_frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);

        char output_path[128];
        sprintf(output_path, "results/CUDA_240p/frame_%02d.png", i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, h_frame_color, frame_w * frame_c);
        
        // Update template for next frame
        update_template(d_frame_gray, frame_w, h_template_gray, template_w, template_h, best_x, best_y);
        gpuErrchk( cudaMemcpy(d_template_gray, h_template_gray, template_w * template_h, cudaMemcpyHostToDevice) );
        
        // Recompute template norm
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)h_template_gray[j] * h_template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);
        
        stbi_image_free(h_frame_color);
        gpuErrchk( cudaFree(d_frame_color) );
        gpuErrchk( cudaFree(d_frame_gray) );
        gpuErrchk( cudaFree(d_similarity_map) );
    }
    
    gpuErrchk( cudaFree(d_template_gray) );
    free(h_template_gray);


    return 0;
}


void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
        }
    }
    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
        }
    }
}

void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y) {
    cudaMemcpy2D(h_template_gray, template_w, 
                 d_frame_gray + best_y * frame_w + best_x, frame_w,
                 template_w, template_h, cudaMemcpyDeviceToHost);
}


Overwriting CUDA_240p.cu


In [32]:
%%bash
nvcc CUDA_240p.cu -o CUDA_240p -Wno-deprecated-gpu-targets "-diag-suppress=550"

In [33]:
%%bash
nvprof ./CUDA_240p

==486845== NVPROF is profiling process 486845, command: ./CUDA_240p


Starting CUDA 240p object tracker (WITH SHARED TEMPLATE VERSION)
Template loaded successfully (87 x 118).
Frame 0: Best match found at (x=138, y=46) with similarity: 0.999621
Frame 1: Best match found at (x=147, y=49) with similarity: 0.976051
Frame 2: Best match found at (x=160, y=47) with similarity: 0.978532
Frame 3: Best match found at (x=180, y=52) with similarity: 0.945356
Frame 4: Best match found at (x=170, y=51) with similarity: 0.973051
Frame 5: Best match found at (x=151, y=47) with similarity: 0.976266
Frame 6: Best match found at (x=129, y=50) with similarity: 0.963636
Frame 7: Best match found at (x=125, y=51) with similarity: 0.971881


==486845== Profiling application: ./CUDA_240p
==486845== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.69%  6.5247ms         8  815.59us  815.19us  816.09us  find_best_match_kernel(unsigned char const *, int, int, unsigned char const *, int, int, double, double*)
                    6.54%  465.27us        17  27.369us  1.8880us  62.047us  [CUDA memcpy HtoD]
                    0.81%  57.760us        33  1.7500us  1.3120us  2.6560us  [CUDA memcpy DtoH]
                    0.64%  45.312us        16  2.8320us  2.5600us  3.2320us  find_max_kernel(double const *, int, double*, int*)
                    0.32%  22.656us         9  2.5170us  2.4320us  2.8160us  color_to_grayscale_kernel(unsigned char const *, unsigned char*, int, int, int)
      API calls:   97.74%  908.24ms        58  15.659ms  6.7920us  905.99ms  cudaMalloc
                    1.19%  11.076ms        42  263.71us  17.967us  765.32us  cudaMemcpy
      

In [34]:
%%bash
nsys profile  -o CUDA_240p ./CUDA_240p

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



Starting CUDA 240p object tracker (WITH SHARED TEMPLATE VERSION)
Template loaded successfully (87 x 118).
Frame 0: Best match found at (x=138, y=46) with similarity: 0.999621
Frame 1: Best match found at (x=147, y=49) with similarity: 0.976051
Frame 2: Best match found at (x=160, y=47) with similarity: 0.978532
Frame 3: Best match found at (x=180, y=52) with similarity: 0.945356
Frame 4: Best match found at (x=170, y=51) with similarity: 0.973051
Frame 5: Best match found at (x=151, y=47) with similarity: 0.976266
Frame 6: Best match found at (x=129, y=50) with similarity: 0.963636
Frame 7: Best match found at (x=125, y=51) with similarity: 0.971881


Failed to create '/home/jupyter-dani_ughoc@dlsu.ed-85ec4/CUDA_240p.nsys-rep': File exists.
Use `--force-overwrite true` to overwrite existing files.


Generating '/tmp/nsys-report-ffb6.qdstrm'
[1/1] [========================100%] nsys-report-b537.nsys-rep
Generated:
	/tmp/nsys-report-b537.nsys-rep


# CUDA 240p w/o Shared Memory Template

In [9]:
%%writefile CUDA_240p_NO_SHARED_TEMP.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

//this function is just for error checking
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define BLOCK_SIZE 16
#define REDUCTION_THREADS 256

__global__ 
void color_to_grayscale_kernel(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        //index for the gray_img
        int gray_idx = y * width + x;
        //index for color_img
        int color_idx = gray_idx * channels; 

        unsigned char r = color_img[color_idx];
        unsigned char g = color_img[color_idx + 1];
        unsigned char b = color_img[color_idx + 2];

        gray_img[gray_idx] = (unsigned char)(0.299f * r + 0.587f * g + 0.114f * b);
    }
}

__global__ 
void find_best_match_kernel(const unsigned char *frame_gray, int frame_w, int frame_h,
                            const unsigned char *template_gray, int template_w, int template_h,
                            double template_norm, double *similarity_map) {

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x <= frame_w - template_w && y <= frame_h - template_h) {
        double dot_product = 0.0;
        double patch_norm_sq = 0.0;

        for (int ty = 0; ty < template_h; ++ty) {
            for (int tx = 0; tx < template_w; ++tx) {
                int frame_idx = (y + ty) * frame_w + (x + tx);
                int template_idx = ty * template_w + tx;
                
                unsigned char frame_val = frame_gray[frame_idx];
                unsigned char template_val = template_gray[template_idx];
                
                dot_product += (double)frame_val * template_val;
                patch_norm_sq += (double)frame_val * frame_val;
            }
        }

        double patch_norm = sqrt(patch_norm_sq);
        double similarity;
        if (patch_norm == 0 || template_norm == 0) {
            similarity = 0.0f;
        } else {
            similarity = (double)(dot_product / (patch_norm * template_norm));
        }

        similarity_map[y * (frame_w - template_w + 1) + x] = similarity;
    }
}
__global__
void find_max_kernel(const double *similarity_map, int map_size, 
                     double *block_max_vals, int *block_max_idxs) {
    __shared__ double shared_vals[REDUCTION_THREADS];
    __shared__ int shared_idxs[REDUCTION_THREADS];

    int tid = threadIdx.x; //Thread's position within block 
    int idx = blockIdx.x * blockDim.x + threadIdx.x; //Index in the similarity map where the thread will get the value

    double my_val = -2.0;
    int my_idx = 0;
    
    if (idx < map_size) {
        my_val = similarity_map[idx]; //get the id's value in the similarity map
        my_idx = idx;
    }

    shared_vals[tid] = my_val; // write value in the shared memory; shared memory is shared within the block
    shared_idxs[tid] = my_idx; // store the global position across all blocks 
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s/=2) {
        if (tid < s) {
            if (shared_vals[tid + s] > shared_vals[tid]) { //example: compare value at index 0 to value at index 128, then store which is higher
                shared_vals[tid] = shared_vals[tid + s];
                shared_idxs[tid] = shared_idxs[tid + s];
            }
        }
        __syncthreads();
    }
    if (tid == 0) {
        block_max_vals[blockIdx.x] = shared_vals[0]; // best is at index 0
        block_max_idxs[blockIdx.x] = shared_idxs[0];
    }
}

void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {
    printf("Starting CUDA 240p object tracker (NO SHARED TEMPLATE VERSION)\n");

    int template_w, template_h, template_c;
    unsigned char *h_template_color = stbi_load("data/template_240.jpg", &template_w, &template_h, &template_c, 3);
    if (h_template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load template image 'data/template_240.jpg'. Check the file path.\n");
        return 1;
    }
    template_c = 3; 
    printf("Template loaded successfully (%d x %d).\n", template_w, template_h);


    unsigned char *d_template_color, *d_template_gray;

    // allocate space in the GPU for the template color and grayscale
    gpuErrchk( cudaMalloc((void**)&d_template_color, template_w * template_h * 3) );
    gpuErrchk( cudaMalloc((void**)&d_template_gray, template_w * template_h) );

    //Memcpy(dst,src)
    gpuErrchk( cudaMemcpy(d_template_color, h_template_color, template_w * template_h * 3, cudaMemcpyHostToDevice) );
    
    dim3 grid_template((template_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (template_h + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block_template(BLOCK_SIZE, BLOCK_SIZE);
    color_to_grayscale_kernel<<<grid_template, block_template>>>(d_template_color, d_template_gray, template_w, template_h, 3);
    gpuErrchk( cudaPeekAtLastError() );
    
    gpuErrchk( cudaFree(d_template_color) );
    stbi_image_free(h_template_color);

    // Calculate template norm on CPU
    unsigned char *h_template_gray = (unsigned char *)malloc(template_w * template_h);
    gpuErrchk( cudaMemcpy(h_template_gray, d_template_gray, template_w * template_h, cudaMemcpyDeviceToHost) );

    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) template_norm_sq += (double)h_template_gray[i] * h_template_gray[i];
    double template_norm = sqrt(template_norm_sq);

    for (int i = 0; i < 8; ++i) {
        char frame_path[128];
        sprintf(frame_path, "data/images240p/%d.jpg", i);

        int frame_w, frame_h, frame_c;
        unsigned char *h_frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 3);
        if (h_frame_color == NULL) {
            fprintf(stderr, "ERROR: Could not load frame %d from '%s'. Skipping.\n", i, frame_path);
            continue;
        }
        frame_c = 3;

        unsigned char *d_frame_color, *d_frame_gray;
        double *d_similarity_map; //for storing the similarity scores
        int map_w = frame_w - template_w + 1;
        int map_h = frame_h - template_h + 1;

        //allocate to global memory
        gpuErrchk( cudaMalloc((void**)&d_frame_color, frame_w * frame_h * frame_c) );
        gpuErrchk( cudaMalloc((void**)&d_frame_gray, frame_w * frame_h) );
        gpuErrchk( cudaMalloc((void**)&d_similarity_map, map_w * map_h * sizeof(double)) );
        gpuErrchk( cudaMemcpy(d_frame_color, h_frame_color, frame_w * frame_h * frame_c, cudaMemcpyHostToDevice) );


        dim3 grid_dim_gray( (frame_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (frame_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_gray(BLOCK_SIZE, BLOCK_SIZE);
        color_to_grayscale_kernel<<<grid_dim_gray, block_dim_gray>>>(d_frame_color, d_frame_gray, frame_w, frame_h, frame_c);
        gpuErrchk( cudaPeekAtLastError() );

        //smaller dimension since we process only valid template positions
        dim3 grid_dim_match( (map_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (map_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_match(BLOCK_SIZE, BLOCK_SIZE);
        
        find_best_match_kernel<<<grid_dim_match, block_dim_match>>>(d_frame_gray, frame_w, frame_h, d_template_gray, template_w, template_h, template_norm, d_similarity_map);
        gpuErrchk( cudaPeekAtLastError() );

        // Two-pass reduction
        int map_size = map_w * map_h;
        int threads = REDUCTION_THREADS;
        int blocks = (map_size + threads - 1) / threads;

        double *d_block_max;
        int *d_block_idx;
        gpuErrchk( cudaMalloc((void**)&d_block_max, blocks * sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_block_idx, blocks * sizeof(int)) );

        // First pass: get the best loc within each block
        find_max_kernel<<<blocks, threads>>>(d_similarity_map, map_size, d_block_max, d_block_idx);
        gpuErrchk( cudaPeekAtLastError() );

        // Second pass: get the final best loc among all the blocks 
        int best_x, best_y;
        double max_similarity;
        int global_idx;
        
        double *d_final_max;
        int *d_final_idx;
        gpuErrchk( cudaMalloc((void**)&d_final_max, sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_final_idx, sizeof(int)) );

        find_max_kernel<<<1, threads>>>(d_block_max, blocks, d_final_max, d_final_idx);//send 1 block of 256 threads because we are only processing ~164 blocks
        gpuErrchk( cudaPeekAtLastError() );

        int winning_block;//the block that holds the best location
        gpuErrchk( cudaMemcpy(&max_similarity, d_final_max, sizeof(double), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&winning_block, d_final_idx, sizeof(int), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&global_idx, &d_block_idx[winning_block], sizeof(int), cudaMemcpyDeviceToHost) );

        // same sa C
        best_x = global_idx % map_w;
        best_y = global_idx / map_w;

        gpuErrchk( cudaFree(d_block_max) );
        gpuErrchk( cudaFree(d_block_idx) );
        gpuErrchk( cudaFree(d_final_max) );
        gpuErrchk( cudaFree(d_final_idx) );

        printf("Frame %d: Best match found at (x=%d, y=%d) with similarity: %f\n", i, best_x, best_y, max_similarity);

        draw_rectangle(h_frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);

        char output_path[128];
        sprintf(output_path, "results/CUDA_240p_NO_SHARED/frame_%02d.png", i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, h_frame_color, frame_w * frame_c);
        
        // Update template for next frame
        update_template(d_frame_gray, frame_w, h_template_gray, template_w, template_h, best_x, best_y);
        gpuErrchk( cudaMemcpy(d_template_gray, h_template_gray, template_w * template_h, cudaMemcpyHostToDevice) );
        
        // Recompute template norm
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)h_template_gray[j] * h_template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);
        
        stbi_image_free(h_frame_color);
        gpuErrchk( cudaFree(d_frame_color) );
        gpuErrchk( cudaFree(d_frame_gray) );
        gpuErrchk( cudaFree(d_similarity_map) );
    }
    
    gpuErrchk( cudaFree(d_template_gray) );
    free(h_template_gray);


    return 0;
}


void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
        }
    }
    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
        }
    }
}

void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y) {
    cudaMemcpy2D(h_template_gray, template_w, 
                 d_frame_gray + best_y * frame_w + best_x, frame_w,
                 template_w, template_h, cudaMemcpyDeviceToHost);
}


Writing CUDA_240p_NO_SHARED_TEMP.cu


In [10]:
%%bash
nvcc CUDA_240p_NO_SHARED_TEMP.cu -o CUDA_240p_NO_SHARED_TEMP -Wno-deprecated-gpu-targets "-diag-suppress=550"

In [11]:
%%bash
nvprof ./CUDA_240p_NO_SHARED_TEMP

==485904== NVPROF is profiling process 485904, command: ./CUDA_240p_NO_SHARED_TEMP


Starting CUDA 240p object tracker (NO SHARED TEMPLATE VERSION)
Template loaded successfully (87 x 118).
Frame 0: Best match found at (x=138, y=46) with similarity: 0.999621
Frame 1: Best match found at (x=147, y=49) with similarity: 0.976051
Frame 2: Best match found at (x=160, y=47) with similarity: 0.978532
Frame 3: Best match found at (x=180, y=52) with similarity: 0.945356
Frame 4: Best match found at (x=170, y=51) with similarity: 0.973051
Frame 5: Best match found at (x=151, y=47) with similarity: 0.976266
Frame 6: Best match found at (x=129, y=50) with similarity: 0.963636
Frame 7: Best match found at (x=125, y=51) with similarity: 0.971881


==485904== Profiling application: ./CUDA_240p_NO_SHARED_TEMP
==485904== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.69%  6.7184ms         8  839.80us  839.19us  840.38us  find_best_match_kernel(unsigned char const *, int, int, unsigned char const *, int, int, double, double*)
                    6.47%  474.20us        17  27.894us  1.8880us  61.407us  [CUDA memcpy HtoD]
                    0.91%  66.973us        33  2.0290us  1.3120us  3.9360us  [CUDA memcpy DtoH]
                    0.62%  45.312us        16  2.8320us  2.5600us  3.2320us  find_max_kernel(double const *, int, double*, int*)
                    0.31%  22.656us         9  2.5170us  2.4000us  3.0080us  color_to_grayscale_kernel(unsigned char const *, unsigned char*, int, int, int)
      API calls:   97.47%  1.09525s        58  18.884ms  7.0110us  1.09266s  cudaMalloc
                    1.16%  13.041ms        42  310.50us  19.104us  925.54us  cu

In [12]:
%%bash
nsys profile  -o CUDA_240p_NO_SHARED_TEMP ./CUDA_240p_NO_SHARED_TEMP

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



Starting CUDA 240p object tracker (NO SHARED TEMPLATE VERSION)
Template loaded successfully (87 x 118).
Frame 0: Best match found at (x=138, y=46) with similarity: 0.999621
Frame 1: Best match found at (x=147, y=49) with similarity: 0.976051
Frame 2: Best match found at (x=160, y=47) with similarity: 0.978532
Frame 3: Best match found at (x=180, y=52) with similarity: 0.945356
Frame 4: Best match found at (x=170, y=51) with similarity: 0.973051
Frame 5: Best match found at (x=151, y=47) with similarity: 0.976266
Frame 6: Best match found at (x=129, y=50) with similarity: 0.963636
Frame 7: Best match found at (x=125, y=51) with similarity: 0.971881
Generating '/tmp/nsys-report-1f3a.qdstrm'
[1/1] [========================100%] CUDA_240p_NO_SHARED_TEMP.nsys-rep
Generated:
	/home/jupyter-dani_ughoc@dlsu.ed-85ec4/CUDA_240p_NO_SHARED_TEMP.nsys-rep


# CUDA 480p with shared memory

In [13]:
%%writefile CUDA_480p_with_SHARED.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define BLOCK_SIZE 16
#define REDUCTION_THREADS 256

__global__ 
void color_to_grayscale_kernel(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int gray_idx = y * width + x;
        int color_idx = gray_idx * channels; 

        unsigned char r = color_img[color_idx];
        unsigned char g = color_img[color_idx + 1];
        unsigned char b = color_img[color_idx + 2];

        gray_img[gray_idx] = (unsigned char)(0.299f * r + 0.587f * g + 0.114f * b);
    }
}

__global__ 
void find_best_match_kernel(const unsigned char *frame_gray, int frame_w, int frame_h,
                            const unsigned char *template_gray, int template_w, int template_h,
                            double template_norm, double *similarity_map) {

    extern __shared__ unsigned char shared_template[];

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    int tid = threadIdx.y * blockDim.x + threadIdx.x;
    int block_threads = blockDim.x * blockDim.y;

    // Load template into shared memory cooperatively
    int template_size = template_w * template_h;
    for (int i = tid; i < template_size; i += block_threads) {
        shared_template[i] = template_gray[i];
    }
    __syncthreads();

    if (x <= frame_w - template_w && y <= frame_h - template_h) {
        double dot_product = 0.0;
        double patch_norm_sq = 0.0;

        for (int ty = 0; ty < template_h; ++ty) {
            for (int tx = 0; tx < template_w; ++tx) {
                int frame_idx = (y + ty) * frame_w + (x + tx);
                int template_idx = ty * template_w + tx;

                unsigned char frame_val = frame_gray[frame_idx];
                unsigned char template_val = shared_template[template_idx];

                dot_product += (double)frame_val * template_val;
                patch_norm_sq += (double)frame_val * frame_val;
            }
        }

        double patch_norm = sqrt(patch_norm_sq);
        double similarity;
        if (patch_norm == 0 || template_norm == 0) {
            similarity = 0.0f;
        } else {
            similarity = (double)(dot_product / (patch_norm * template_norm));
        }
        similarity_map[y * (frame_w - template_w + 1) + x] = similarity;
    }
}

__global__
void find_max_kernel(const double *similarity_map, const int *input_idxs, int map_size, 
                     double *block_max_vals, int *block_max_idxs, bool first_pass) {
    __shared__ double shared_vals[REDUCTION_THREADS];
    __shared__ int shared_idxs[REDUCTION_THREADS];

    int tid = threadIdx.x; //local thread index sa block
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // index in the similarity map

    double my_val = -2.0;
    int my_idx = 0;

    if (idx < map_size) {
        my_val = similarity_map[idx];
        // For first pass, use idx directly; for subsequent passes, use the stored index
        if (first_pass) {
            my_idx = idx;
        } else {
            my_idx = input_idxs[idx];//index from the original similiarity map and not from the first pass
        }
    }

    shared_vals[tid] = my_val;  //write value in the shared memory; shared memory is shared within the block
    shared_idxs[tid] = my_idx;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s/=2) {
        if (tid < s) {
            if (shared_vals[tid + s] > shared_vals[tid]) { //example: compare value at index 0 to value at index 128, then store which is higher
                shared_vals[tid] = shared_vals[tid + s];
                shared_idxs[tid] = shared_idxs[tid + s];
            }
        }
        __syncthreads();
    }

    if (tid == 0) {
        block_max_vals[blockIdx.x] = shared_vals[0]; // best within the block is at index 0
        block_max_idxs[blockIdx.x] = shared_idxs[0];
    }
}

void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {

    printf("Starting CUDA 480p object tracker (SHARED TEMPLATE VERSION)\n");

    int template_w, template_h, template_c;
    unsigned char *h_template_color = stbi_load("data/template_480.jpg", &template_w, &template_h, &template_c, 3);
    if (h_template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load template image 'data/template_480.jpg'. Check the file path.\n");
        return 1;
    }
    template_c = 3; 
    printf("Template loaded successfully (%d x %d).\n", template_w, template_h);

 // Upload color template to GPU and convert to grayscale on GPU
    unsigned char *d_template_color, *d_template_gray;
    gpuErrchk( cudaMalloc((void**)&d_template_color, template_w * template_h * 3) );
    gpuErrchk( cudaMalloc((void**)&d_template_gray, template_w * template_h) );
    gpuErrchk( cudaMemcpy(d_template_color, h_template_color, template_w * template_h * 3, cudaMemcpyHostToDevice) );
    
    dim3 grid_template((template_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (template_h + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block_template(BLOCK_SIZE, BLOCK_SIZE);
    color_to_grayscale_kernel<<<grid_template, block_template>>>(d_template_color, d_template_gray, template_w, template_h, 3);
    gpuErrchk( cudaPeekAtLastError() );
    
    gpuErrchk( cudaFree(d_template_color) );
    stbi_image_free(h_template_color);

    // Calculate template norm on CPU
    unsigned char *h_template_gray = (unsigned char *)malloc(template_w * template_h);
    gpuErrchk( cudaMemcpy(h_template_gray, d_template_gray, template_w * template_h, cudaMemcpyDeviceToHost) );

    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) template_norm_sq += (double)h_template_gray[i] * h_template_gray[i];
    double template_norm = sqrt(template_norm_sq);

    for (int i = 0; i < 8; ++i) {
        char frame_path[128];
        sprintf(frame_path, "data/images480p/%d.jpg", i);

        int frame_w, frame_h, frame_c;
        unsigned char *h_frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 3);
        if (h_frame_color == NULL) {
            fprintf(stderr, "ERROR: Could not load frame %d from '%s'. Skipping.\n", i, frame_path);
            continue;
        }
        frame_c = 3;

        unsigned char *d_frame_color, *d_frame_gray;
        double *d_similarity_map;
        int map_w = frame_w - template_w + 1;//854-150=704 +1 = 705
        int map_h = frame_h - template_h + 1;//480-232=249 + 1 =249

        gpuErrchk( cudaMalloc((void**)&d_frame_color, frame_w * frame_h * frame_c) );
        gpuErrchk( cudaMalloc((void**)&d_frame_gray, frame_w * frame_h) );
        gpuErrchk( cudaMalloc((void**)&d_similarity_map, map_w * map_h * sizeof(double)) );
        gpuErrchk( cudaMemcpy(d_frame_color, h_frame_color, frame_w * frame_h * frame_c, cudaMemcpyHostToDevice) );

        dim3 grid_dim_gray( (frame_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (frame_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_gray(BLOCK_SIZE, BLOCK_SIZE);
        color_to_grayscale_kernel<<<grid_dim_gray, block_dim_gray>>>(d_frame_color, d_frame_gray, frame_w, frame_h, frame_c);
        gpuErrchk( cudaPeekAtLastError() );

        dim3 grid_dim_match( (map_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (map_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_match(BLOCK_SIZE, BLOCK_SIZE);
        int shared_mem_size = template_w * template_h * sizeof(unsigned char);

        find_best_match_kernel<<<grid_dim_match, block_dim_match, shared_mem_size>>>(d_frame_gray, frame_w, frame_h, d_template_gray, template_w, template_h, template_norm, d_similarity_map);
        gpuErrchk( cudaPeekAtLastError() );

        // Multi-stage reduction
        //
        int map_size = map_w * map_h; //705*249 = 175,545 possible best loc
        int threads = REDUCTION_THREADS;
        int blocks = (map_size + threads - 1) / threads; //(175,545+256-1)/256 = 686

        double *d_block_max; // will store the best similarity value per block
        int *d_block_idx; // will hold the original full map index of that block's winner
        gpuErrchk( cudaMalloc((void**)&d_block_max, blocks * sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_block_idx, blocks * sizeof(int)) );

        // First pass                                         //NULL because the first pass still has the original global index                
        find_max_kernel<<<blocks, threads>>>(d_similarity_map, NULL, map_size, d_block_max, d_block_idx, true);
        gpuErrchk( cudaPeekAtLastError() );                                      // where the block will store the local best loc and its original index        
        //after first past, we have 686 best in each block
            
        // Continue reducing until we have a single result
        double *d_curr_vals = d_block_max;//points to the first pass results
        int *d_curr_idxs = d_block_idx;// points to the first pass results original indexes
        int curr_size = blocks; //686

        while (curr_size > 1) {
            int next_blocks = (curr_size + threads - 1) / threads;//(686+256-1)/256=3

            double *d_next_vals;//will hold the 3 best loc after 2nd pass
            int *d_next_idxs;
            gpuErrchk( cudaMalloc((void**)&d_next_vals, next_blocks * sizeof(double)) );
            gpuErrchk( cudaMalloc((void**)&d_next_idxs, next_blocks * sizeof(int)) );
                            //3,256
            find_max_kernel<<<next_blocks, threads>>>(d_curr_vals, d_curr_idxs, curr_size, d_next_vals, d_next_idxs, false);
            gpuErrchk( cudaPeekAtLastError() );

            if (d_curr_vals != d_block_max) { //free only the arrays we created inside the loop
                gpuErrchk( cudaFree(d_curr_vals) );
                gpuErrchk( cudaFree(d_curr_idxs) );
            }

            d_curr_vals = d_next_vals;//points to the second pass result which as 3 elements
            d_curr_idxs = d_next_idxs;
            curr_size = next_blocks;//3
        }

        // Copy final result
        int best_x, best_y;
        double max_similarity;
        int global_idx;

        gpuErrchk( cudaMemcpy(&max_similarity, d_curr_vals, sizeof(double), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&global_idx, d_curr_idxs, sizeof(int), cudaMemcpyDeviceToHost) );

        // CPU computes x,y
        best_x = global_idx % map_w;
        best_y = global_idx / map_w;

        if (d_curr_vals != d_block_max) {//if d_curr_vals still points to the 1-element or 3-element
            gpuErrchk( cudaFree(d_curr_vals) );
            gpuErrchk( cudaFree(d_curr_idxs) );
        }
        // free first-pass arrays
        gpuErrchk( cudaFree(d_block_max) );
        gpuErrchk( cudaFree(d_block_idx) );


        printf("Frame %d: Best match found at (x=%d, y=%d) with similarity: %f\n", i, best_x, best_y, max_similarity);

        draw_rectangle(h_frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);

        char output_path[128];
        sprintf(output_path, "results/CUDA_480p_withshared_memory/frame_%02d.png", i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, h_frame_color, frame_w * frame_c);

        // Update template for next frame
        update_template(d_frame_gray, frame_w, h_template_gray, template_w, template_h, best_x, best_y);
        gpuErrchk( cudaMemcpy(d_template_gray, h_template_gray, template_w * template_h, cudaMemcpyHostToDevice) );

        // Recompute template norm
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)h_template_gray[j] * h_template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);

        stbi_image_free(h_frame_color);
        gpuErrchk( cudaFree(d_frame_color) );
        gpuErrchk( cudaFree(d_frame_gray) );
        gpuErrchk( cudaFree(d_similarity_map) );
    }

    gpuErrchk( cudaFree(d_template_gray) );
    free(h_template_gray);

    return 0;
}


void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
        }
    }
    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
        }
    }
}

void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y) {
    cudaMemcpy2D(h_template_gray, template_w, 
                 d_frame_gray + best_y * frame_w + best_x, frame_w,
                 template_w, template_h, cudaMemcpyDeviceToHost);
}


Writing CUDA_480p_with_SHARED.cu


In [14]:
%%bash
nvcc CUDA_480p_with_SHARED.cu -o CUDA_480p_with_SHARED -Wno-deprecated-gpu-targets "-diag-suppress=550"

In [15]:
%%bash
nvprof ./CUDA_480p_with_SHARED

==486039== NVPROF is profiling process 486039, command: ./CUDA_480p_with_SHARED


Starting CUDA 480p object tracker (SHARED TEMPLATE VERSION)
Template loaded successfully (150 x 232).
Frame 0: Best match found at (x=289, y=90) with similarity: 0.999808
Frame 1: Best match found at (x=300, y=106) with similarity: 0.973546
Frame 2: Best match found at (x=329, y=105) with similarity: 0.972979
Frame 3: Best match found at (x=372, y=114) with similarity: 0.948270
Frame 4: Best match found at (x=351, y=112) with similarity: 0.975688
Frame 5: Best match found at (x=312, y=103) with similarity: 0.976127
Frame 6: Best match found at (x=268, y=107) with similarity: 0.963581
Frame 7: Best match found at (x=261, y=110) with similarity: 0.970502


==486039== Profiling application: ./CUDA_480p_with_SHARED
==486039== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   97.36%  77.070ms         8  9.6338ms  9.6297ms  9.6372ms  find_best_match_kernel(unsigned char const *, int, int, unsigned char const *, int, int, double, double*)
                    2.38%  1.8862ms        17  110.95us  5.1840us  255.36us  [CUDA memcpy HtoD]
                    0.11%  88.415us        24  3.6830us  2.6560us  5.6960us  find_max_kernel(double const *, int const *, int, double*, int*, bool)
                    0.10%  76.800us        25  3.0720us  1.3120us  8.7040us  [CUDA memcpy DtoH]
                    0.05%  40.800us         9  4.5330us  3.4560us  4.9280us  color_to_grayscale_kernel(unsigned char const *, unsigned char*, int, int, int)
      API calls:   86.37%  620.60ms        74  8.3864ms  5.5790us  614.03ms  cudaMalloc
                   11.42%  82.075ms        74  1.1091ms  6.2870

In [16]:
%%bash
nsys profile  -o CUDA_480p_with_SHARED ./CUDA_480p_with_SHARED

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



Starting CUDA 480p object tracker (SHARED TEMPLATE VERSION)
Template loaded successfully (150 x 232).
Frame 0: Best match found at (x=289, y=90) with similarity: 0.999808
Frame 1: Best match found at (x=300, y=106) with similarity: 0.973546
Frame 2: Best match found at (x=329, y=105) with similarity: 0.972979
Frame 3: Best match found at (x=372, y=114) with similarity: 0.948270
Frame 4: Best match found at (x=351, y=112) with similarity: 0.975688
Frame 5: Best match found at (x=312, y=103) with similarity: 0.976127
Frame 6: Best match found at (x=268, y=107) with similarity: 0.963581
Frame 7: Best match found at (x=261, y=110) with similarity: 0.970502
Generating '/tmp/nsys-report-ddfa.qdstrm'
[1/1] [========================100%] CUDA_480p_with_SHARED.nsys-rep
Generated:
	/home/jupyter-dani_ughoc@dlsu.ed-85ec4/CUDA_480p_with_SHARED.nsys-rep


# CUDA 480p w/o shared memory template

In [17]:
%%writefile CUDA_480p.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define BLOCK_SIZE 16
#define REDUCTION_THREADS 256

__global__ 
void color_to_grayscale_kernel(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int gray_idx = y * width + x;
        int color_idx = gray_idx * channels; 

        unsigned char r = color_img[color_idx];
        unsigned char g = color_img[color_idx + 1];
        unsigned char b = color_img[color_idx + 2];

        gray_img[gray_idx] = (unsigned char)(0.299f * r + 0.587f * g + 0.114f * b);
    }
}

__global__ 
void find_best_match_kernel(const unsigned char *frame_gray, int frame_w, int frame_h,
                            const unsigned char *template_gray, int template_w, int template_h,
                            double template_norm, double *similarity_map) {

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x <= frame_w - template_w && y <= frame_h - template_h) {
        double dot_product = 0.0;
        double patch_norm_sq = 0.0;

        for (int ty = 0; ty < template_h; ++ty) {
            for (int tx = 0; tx < template_w; ++tx) {
                int frame_idx = (y + ty) * frame_w + (x + tx);
                int template_idx = ty * template_w + tx;

                unsigned char frame_val = frame_gray[frame_idx];
                unsigned char template_val = template_gray[template_idx];

                dot_product += (double)frame_val * template_val;
                patch_norm_sq += (double)frame_val * frame_val;
            }
        }

        double patch_norm = sqrt(patch_norm_sq);
        double similarity;
        if (patch_norm == 0 || template_norm == 0) {
            similarity = 0.0f;
        } else {
            similarity = (double)(dot_product / (patch_norm * template_norm));
        }
        similarity_map[y * (frame_w - template_w + 1) + x] = similarity;
    }
}

__global__
void find_max_kernel(const double *similarity_map, const int *input_idxs, int map_size, 
                     double *block_max_vals, int *block_max_idxs, bool first_pass) {
    __shared__ double shared_vals[REDUCTION_THREADS];
    __shared__ int shared_idxs[REDUCTION_THREADS];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    double my_val = -2.0;
    int my_idx = 0;

    if (idx < map_size) {
        my_val = similarity_map[idx];
        // For first pass, use idx directly; for subsequent passes, use the stored index
        if (first_pass) {
            my_idx = idx;
        } else {
            my_idx = input_idxs[idx];
        }
    }

    shared_vals[tid] = my_val;
    shared_idxs[tid] = my_idx;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s/=2) {
        if (tid < s) {
            if (shared_vals[tid + s] > shared_vals[tid]) { //example: compare value at index 0 to value at index 128, then store which is higher
                shared_vals[tid] = shared_vals[tid + s];
                shared_idxs[tid] = shared_idxs[tid + s];
            }
        }
        __syncthreads();
    }

    if (tid == 0) {
        block_max_vals[blockIdx.x] = shared_vals[0];
        block_max_idxs[blockIdx.x] = shared_idxs[0];
    }
}

void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {


    printf("Starting CUDA 480p object tracker (NO SHARED TEMPLATE VERSION)\n");

    int template_w, template_h, template_c;
    unsigned char *h_template_color = stbi_load("data/template_480.jpg", &template_w, &template_h, &template_c, 3);
    if (h_template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load template image 'data/template_480.jpg'. Check the file path.\n");
        return 1;
    }
    template_c = 3; 
    printf("Template loaded successfully (%d x %d).\n", template_w, template_h);

       // Upload color template to GPU and convert to grayscale on GPU
    unsigned char *d_template_color, *d_template_gray;
    gpuErrchk( cudaMalloc((void**)&d_template_color, template_w * template_h * 3) );
    gpuErrchk( cudaMalloc((void**)&d_template_gray, template_w * template_h) );
    gpuErrchk( cudaMemcpy(d_template_color, h_template_color, template_w * template_h * 3, cudaMemcpyHostToDevice) );
    
    dim3 grid_template((template_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (template_h + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block_template(BLOCK_SIZE, BLOCK_SIZE);
    color_to_grayscale_kernel<<<grid_template, block_template>>>(d_template_color, d_template_gray, template_w, template_h, 3);
    gpuErrchk( cudaPeekAtLastError() );
    
    gpuErrchk( cudaFree(d_template_color) );
    stbi_image_free(h_template_color);

    // Calculate template norm on CPU (need to download template once)
    unsigned char *h_template_gray = (unsigned char *)malloc(template_w * template_h);
    gpuErrchk( cudaMemcpy(h_template_gray, d_template_gray, template_w * template_h, cudaMemcpyDeviceToHost) );

    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) template_norm_sq += (double)h_template_gray[i] * h_template_gray[i];
    double template_norm = sqrt(template_norm_sq);

    for (int i = 0; i < 8; ++i) {
        char frame_path[128];
        sprintf(frame_path, "data/images480p/%d.jpg", i);

        int frame_w, frame_h, frame_c;
        unsigned char *h_frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 3);
        if (h_frame_color == NULL) {
            fprintf(stderr, "ERROR: Could not load frame %d from '%s'. Skipping.\n", i, frame_path);
            continue;
        }
        frame_c = 3;

        unsigned char *d_frame_color, *d_frame_gray;
        double *d_similarity_map;
        int map_w = frame_w - template_w + 1;
        int map_h = frame_h - template_h + 1;

        gpuErrchk( cudaMalloc((void**)&d_frame_color, frame_w * frame_h * frame_c) );
        gpuErrchk( cudaMalloc((void**)&d_frame_gray, frame_w * frame_h) );
        gpuErrchk( cudaMalloc((void**)&d_similarity_map, map_w * map_h * sizeof(double)) );
        gpuErrchk( cudaMemcpy(d_frame_color, h_frame_color, frame_w * frame_h * frame_c, cudaMemcpyHostToDevice) );


        dim3 grid_dim_gray( (frame_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (frame_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_gray(BLOCK_SIZE, BLOCK_SIZE);
        color_to_grayscale_kernel<<<grid_dim_gray, block_dim_gray>>>(d_frame_color, d_frame_gray, frame_w, frame_h, frame_c);
        gpuErrchk( cudaPeekAtLastError() );

        dim3 grid_dim_match( (map_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (map_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_match(BLOCK_SIZE, BLOCK_SIZE);

        find_best_match_kernel<<<grid_dim_match, block_dim_match>>>(
            d_frame_gray, frame_w, frame_h, d_template_gray, template_w, template_h, template_norm, d_similarity_map);
        gpuErrchk( cudaPeekAtLastError() );

        // Multi-stage reduction
        int map_size = map_w * map_h;
        int threads = REDUCTION_THREADS;
        int blocks = (map_size + threads - 1) / threads;

        double *d_block_max;
        int *d_block_idx;
        gpuErrchk( cudaMalloc((void**)&d_block_max, blocks * sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_block_idx, blocks * sizeof(int)) );

        // First pass: reduce within blocks
        find_max_kernel<<<blocks, threads>>>(d_similarity_map, NULL, map_size, d_block_max, d_block_idx, true);
        gpuErrchk( cudaPeekAtLastError() );

        // Continue reducing until we have a single result
        double *d_curr_vals = d_block_max;
        int *d_curr_idxs = d_block_idx;
        int curr_size = blocks;

        while (curr_size > 1) {
            int next_blocks = (curr_size + threads - 1) / threads;

            double *d_next_vals;
            int *d_next_idxs;
            gpuErrchk( cudaMalloc((void**)&d_next_vals, next_blocks * sizeof(double)) );
            gpuErrchk( cudaMalloc((void**)&d_next_idxs, next_blocks * sizeof(int)) );

            find_max_kernel<<<next_blocks, threads>>>(d_curr_vals, d_curr_idxs, curr_size, d_next_vals, d_next_idxs, false);
            gpuErrchk( cudaPeekAtLastError() );

            // Free previous stage (except the first one which is d_block_max)
            if (d_curr_vals != d_block_max) {
                gpuErrchk( cudaFree(d_curr_vals) );
                gpuErrchk( cudaFree(d_curr_idxs) );
            }

            d_curr_vals = d_next_vals;
            d_curr_idxs = d_next_idxs;
            curr_size = next_blocks;
        }

        // Copy final result
        int best_x, best_y;
        double max_similarity;
        int global_idx;

        gpuErrchk( cudaMemcpy(&max_similarity, d_curr_vals, sizeof(double), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&global_idx, d_curr_idxs, sizeof(int), cudaMemcpyDeviceToHost) );

        // CPU computes x,y
        best_x = global_idx % map_w;
        best_y = global_idx / map_w;

        // Free reduction buffers
        gpuErrchk( cudaFree(d_block_max) );
        gpuErrchk( cudaFree(d_block_idx) );
        if (d_curr_vals != d_block_max) {
            gpuErrchk( cudaFree(d_curr_vals) );
            gpuErrchk( cudaFree(d_curr_idxs) );
        }



        printf("Frame %d: Best match found at (x=%d, y=%d) with similarity: %f\n", i, best_x, best_y, max_similarity);

        draw_rectangle(h_frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);

        char output_path[128];
        sprintf(output_path, "results/CUDA_480p/frame_%02d.png", i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, h_frame_color, frame_w * frame_c);

        // Update template for next frame
        update_template(d_frame_gray, frame_w, h_template_gray, template_w, template_h, best_x, best_y);
        gpuErrchk( cudaMemcpy(d_template_gray, h_template_gray, template_w * template_h, cudaMemcpyHostToDevice) );

        // Recompute template norm
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)h_template_gray[j] * h_template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);

        stbi_image_free(h_frame_color);
        gpuErrchk( cudaFree(d_frame_color) );
        gpuErrchk( cudaFree(d_frame_gray) );
        gpuErrchk( cudaFree(d_similarity_map) );
    }

    gpuErrchk( cudaFree(d_template_gray) );
    free(h_template_gray);

    return 0;
}


void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
        }
    }
    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
        }
    }
}

void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y) {
    cudaMemcpy2D(h_template_gray, template_w, 
                 d_frame_gray + best_y * frame_w + best_x, frame_w,
                 template_w, template_h, cudaMemcpyDeviceToHost);
}


Writing CUDA_480p.cu


In [18]:
%%bash
nvcc CUDA_480p.cu -o CUDA_480p -Wno-deprecated-gpu-targets "-diag-suppress=550"

In [19]:
%%bash
nvprof ./CUDA_480p

==486178== NVPROF is profiling process 486178, command: ./CUDA_480p


Starting CUDA 480p object tracker (NO SHARED TEMPLATE VERSION)
Template loaded successfully (150 x 232).
Frame 0: Best match found at (x=289, y=90) with similarity: 0.999808
Frame 1: Best match found at (x=300, y=106) with similarity: 0.973546
Frame 2: Best match found at (x=329, y=105) with similarity: 0.972979
Frame 3: Best match found at (x=372, y=114) with similarity: 0.948270
Frame 4: Best match found at (x=351, y=112) with similarity: 0.975688
Frame 5: Best match found at (x=312, y=103) with similarity: 0.976127
Frame 6: Best match found at (x=268, y=107) with similarity: 0.963581
Frame 7: Best match found at (x=261, y=110) with similarity: 0.970502


==486178== Profiling application: ./CUDA_480p
==486178== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   96.49%  73.931ms         8  9.2413ms  9.2245ms  9.2644ms  find_best_match_kernel(unsigned char const *, int, int, unsigned char const *, int, int, double, double*)
                    3.21%  2.4606ms        17  144.74us  5.1510us  332.86us  [CUDA memcpy HtoD]
                    0.13%  103.32us        25  4.1320us  1.3440us  11.072us  [CUDA memcpy DtoH]
                    0.12%  88.800us        24  3.7000us  2.6560us  5.7600us  find_max_kernel(double const *, int const *, int, double*, int*, bool)
                    0.05%  40.384us         9  4.4870us  2.7840us  4.9920us  color_to_grayscale_kernel(unsigned char const *, unsigned char*, int, int, int)
      API calls:   89.49%  852.58ms        74  11.521ms  5.8250us  844.91ms  cudaMalloc
                    8.39%  79.955ms        74  1.0805ms  4.9840us  9.2636ms

In [20]:
%%bash
nsys profile  -o CUDA_480p ./CUDA_480p

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



Starting CUDA 480p object tracker (NO SHARED TEMPLATE VERSION)
Template loaded successfully (150 x 232).
Frame 0: Best match found at (x=289, y=90) with similarity: 0.999808
Frame 1: Best match found at (x=300, y=106) with similarity: 0.973546
Frame 2: Best match found at (x=329, y=105) with similarity: 0.972979
Frame 3: Best match found at (x=372, y=114) with similarity: 0.948270
Frame 4: Best match found at (x=351, y=112) with similarity: 0.975688
Frame 5: Best match found at (x=312, y=103) with similarity: 0.976127
Frame 6: Best match found at (x=268, y=107) with similarity: 0.963581
Frame 7: Best match found at (x=261, y=110) with similarity: 0.970502
Generating '/tmp/nsys-report-f8a9.qdstrm'
[1/1] [========================100%] CUDA_480p.nsys-rep
Generated:
	/home/jupyter-dani_ughoc@dlsu.ed-85ec4/CUDA_480p.nsys-rep


# CUDA 720p No Tilling

In [21]:
%%writefile CUDA_720p.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define BLOCK_SIZE 16
#define REDUCTION_THREADS 256

__global__ 
void color_to_grayscale_kernel(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int gray_idx = y * width + x;
        int color_idx = gray_idx * channels; 

        unsigned char r = color_img[color_idx];
        unsigned char g = color_img[color_idx + 1];
        unsigned char b = color_img[color_idx + 2];

        gray_img[gray_idx] = (unsigned char)(0.299f * r + 0.587f * g + 0.114f * b);
    }
}

__global__ 
void find_best_match_kernel(const unsigned char *frame_gray, int frame_w, int frame_h,
                            const unsigned char *template_gray, int template_w, int template_h,
                            double template_norm, double *similarity_map) {

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x <= frame_w - template_w && y <= frame_h - template_h) {
        double dot_product = 0.0;
        double patch_norm_sq = 0.0;

        for (int ty = 0; ty < template_h; ++ty) {
            for (int tx = 0; tx < template_w; ++tx) {
                int frame_idx = (y + ty) * frame_w + (x + tx);
                int template_idx = ty * template_w + tx;

                unsigned char frame_val = frame_gray[frame_idx];
                unsigned char template_val = template_gray[template_idx];

                dot_product += (double)frame_val * template_val;
                patch_norm_sq += (double)frame_val * frame_val;
            }
        }

        double patch_norm = sqrt(patch_norm_sq);
        double similarity;
        if (patch_norm == 0 || template_norm == 0) {
            similarity = 0.0f;
        } else {
            similarity = (double)(dot_product / (patch_norm * template_norm));
        }
        similarity_map[y * (frame_w - template_w + 1) + x] = similarity;
    }
}

__global__
void find_max_kernel(const double *similarity_map, const int *input_idxs, int map_size, 
                     double *block_max_vals, int *block_max_idxs, bool first_pass) {
    __shared__ double shared_vals[REDUCTION_THREADS];
    __shared__ int shared_idxs[REDUCTION_THREADS];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    double my_val = -2.0;
    int my_idx = 0;

    if (idx < map_size) {
        my_val = similarity_map[idx];
        // For first pass, use idx directly; for subsequent passes, use the stored index
        if (first_pass) {
            my_idx = idx;
        } else {
            my_idx = input_idxs[idx];
        }
    }

    shared_vals[tid] = my_val;
    shared_idxs[tid] = my_idx;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s/=2) {
        if (tid < s) {
            if (shared_vals[tid + s] > shared_vals[tid]) { //example: compare value at index 0 to value at index 128, then store which is higher
                shared_vals[tid] = shared_vals[tid + s];
                shared_idxs[tid] = shared_idxs[tid + s];
            }
        }
        __syncthreads();
    }

    if (tid == 0) {
        block_max_vals[blockIdx.x] = shared_vals[0];
        block_max_idxs[blockIdx.x] = shared_idxs[0];
    }
}

void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {


    printf("Starting CUDA 720p object tracker (NO TILLING VERSION)\n");

    int template_w, template_h, template_c;
    unsigned char *h_template_color = stbi_load("data/template_720.jpg", &template_w, &template_h, &template_c, 3);
    if (h_template_color == NULL) {
        fprintf(stderr, "ERROR: Could not load template image 'data/template_720.jpg'. Check the file path.\n");
        return 1;
    }
    template_c = 3; 
    printf("Template loaded successfully (%d x %d).\n", template_w, template_h);

       // Upload color template to GPU and convert to grayscale on GPU
    unsigned char *d_template_color, *d_template_gray;
    gpuErrchk( cudaMalloc((void**)&d_template_color, template_w * template_h * 3) );
    gpuErrchk( cudaMalloc((void**)&d_template_gray, template_w * template_h) );
    gpuErrchk( cudaMemcpy(d_template_color, h_template_color, template_w * template_h * 3, cudaMemcpyHostToDevice) );
    
    dim3 grid_template((template_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (template_h + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block_template(BLOCK_SIZE, BLOCK_SIZE);
    color_to_grayscale_kernel<<<grid_template, block_template>>>(d_template_color, d_template_gray, template_w, template_h, 3);
    gpuErrchk( cudaPeekAtLastError() );
    
    gpuErrchk( cudaFree(d_template_color) );
    stbi_image_free(h_template_color);

    // Calculate template norm on CPU (need to download template once)
    unsigned char *h_template_gray = (unsigned char *)malloc(template_w * template_h);
    gpuErrchk( cudaMemcpy(h_template_gray, d_template_gray, template_w * template_h, cudaMemcpyDeviceToHost) );

    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) template_norm_sq += (double)h_template_gray[i] * h_template_gray[i];
    double template_norm = sqrt(template_norm_sq);

    for (int i = 0; i < 8; ++i) {
        char frame_path[128];
        sprintf(frame_path, "data/images720p/%d.jpg", i);

        int frame_w, frame_h, frame_c;
        unsigned char *h_frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 3);
        if (h_frame_color == NULL) {
            fprintf(stderr, "ERROR: Could not load frame %d from '%s'. Skipping.\n", i, frame_path);
            continue;
        }
        frame_c = 3;

        unsigned char *d_frame_color, *d_frame_gray;
        double *d_similarity_map;
        int map_w = frame_w - template_w + 1;
        int map_h = frame_h - template_h + 1;

        gpuErrchk( cudaMalloc((void**)&d_frame_color, frame_w * frame_h * frame_c) );
        gpuErrchk( cudaMalloc((void**)&d_frame_gray, frame_w * frame_h) );
        gpuErrchk( cudaMalloc((void**)&d_similarity_map, map_w * map_h * sizeof(double)) );
        gpuErrchk( cudaMemcpy(d_frame_color, h_frame_color, frame_w * frame_h * frame_c, cudaMemcpyHostToDevice) );


        dim3 grid_dim_gray( (frame_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (frame_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_gray(BLOCK_SIZE, BLOCK_SIZE);
        color_to_grayscale_kernel<<<grid_dim_gray, block_dim_gray>>>(d_frame_color, d_frame_gray, frame_w, frame_h, frame_c);
        gpuErrchk( cudaPeekAtLastError() );

        dim3 grid_dim_match( (map_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (map_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_match(BLOCK_SIZE, BLOCK_SIZE);

        find_best_match_kernel<<<grid_dim_match, block_dim_match>>>(
            d_frame_gray, frame_w, frame_h, d_template_gray, template_w, template_h, template_norm, d_similarity_map);
        gpuErrchk( cudaPeekAtLastError() );

        // Multi-stage reduction
        int map_size = map_w * map_h;
        int threads = REDUCTION_THREADS;
        int blocks = (map_size + threads - 1) / threads;

        double *d_block_max;
        int *d_block_idx;
        gpuErrchk( cudaMalloc((void**)&d_block_max, blocks * sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_block_idx, blocks * sizeof(int)) );

        // First pass: reduce within blocks
        find_max_kernel<<<blocks, threads>>>(d_similarity_map, NULL, map_size, d_block_max, d_block_idx, true);
        gpuErrchk( cudaPeekAtLastError() );

        // Continue reducing until we have a single result
        double *d_curr_vals = d_block_max;
        int *d_curr_idxs = d_block_idx;
        int curr_size = blocks;

        while (curr_size > 1) {
            int next_blocks = (curr_size + threads - 1) / threads;

            double *d_next_vals;
            int *d_next_idxs;
            gpuErrchk( cudaMalloc((void**)&d_next_vals, next_blocks * sizeof(double)) );
            gpuErrchk( cudaMalloc((void**)&d_next_idxs, next_blocks * sizeof(int)) );

            find_max_kernel<<<next_blocks, threads>>>(d_curr_vals, d_curr_idxs, curr_size, d_next_vals, d_next_idxs, false);
            gpuErrchk( cudaPeekAtLastError() );

            // Free previous stage (except the first one which is d_block_max)
            if (d_curr_vals != d_block_max) {
                gpuErrchk( cudaFree(d_curr_vals) );
                gpuErrchk( cudaFree(d_curr_idxs) );
            }

            d_curr_vals = d_next_vals;
            d_curr_idxs = d_next_idxs;
            curr_size = next_blocks;
        }

        // Copy final result
        int best_x, best_y;
        double max_similarity;
        int global_idx;

        gpuErrchk( cudaMemcpy(&max_similarity, d_curr_vals, sizeof(double), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&global_idx, d_curr_idxs, sizeof(int), cudaMemcpyDeviceToHost) );

        // CPU computes x,y
        best_x = global_idx % map_w;
        best_y = global_idx / map_w;

        // Free reduction buffers
        gpuErrchk( cudaFree(d_block_max) );
        gpuErrchk( cudaFree(d_block_idx) );
        if (d_curr_vals != d_block_max) {
            gpuErrchk( cudaFree(d_curr_vals) );
            gpuErrchk( cudaFree(d_curr_idxs) );
        }



        printf("Frame %d: Best match found at (x=%d, y=%d) with similarity: %f\n", i, best_x, best_y, max_similarity);

        draw_rectangle(h_frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);

        char output_path[128];
        sprintf(output_path, "results/CUDA_720p/frame_%02d.png", i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, h_frame_color, frame_w * frame_c);

        // Update template for next frame
        update_template(d_frame_gray, frame_w, h_template_gray, template_w, template_h, best_x, best_y);
        gpuErrchk( cudaMemcpy(d_template_gray, h_template_gray, template_w * template_h, cudaMemcpyHostToDevice) );

        // Recompute template norm
        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) {
            template_norm_sq += (double)h_template_gray[j] * h_template_gray[j];
        }
        template_norm = sqrt(template_norm_sq);

        stbi_image_free(h_frame_color);
        gpuErrchk( cudaFree(d_frame_color) );
        gpuErrchk( cudaFree(d_frame_gray) );
        gpuErrchk( cudaFree(d_similarity_map) );
    }

    gpuErrchk( cudaFree(d_template_gray) );
    free(h_template_gray);

    return 0;
}


void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
        }
    }
    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
        }
    }
}

void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y) {
    cudaMemcpy2D(h_template_gray, template_w, 
                 d_frame_gray + best_y * frame_w + best_x, frame_w,
                 template_w, template_h, cudaMemcpyDeviceToHost);
}


Writing CUDA_720p.cu


In [22]:
%%bash
nvcc CUDA_720p.cu -o CUDA_720p -Wno-deprecated-gpu-targets "-diag-suppress=550"

In [23]:
%%bash
nvprof ./CUDA_720p

==486321== NVPROF is profiling process 486321, command: ./CUDA_720p


Starting CUDA 720p object tracker (NO TILLING VERSION)
Template loaded successfully (251 x 351).
Frame 0: Best match found at (x=426, y=131) with similarity: 0.999818
Frame 1: Best match found at (x=453, y=141) with similarity: 0.976094
Frame 2: Best match found at (x=493, y=132) with similarity: 0.977169
Frame 3: Best match found at (x=556, y=148) with similarity: 0.944861
Frame 4: Best match found at (x=526, y=144) with similarity: 0.974873
Frame 5: Best match found at (x=468, y=131) with similarity: 0.974679
Frame 6: Best match found at (x=401, y=139) with similarity: 0.964025
Frame 7: Best match found at (x=389, y=143) with similarity: 0.970891


==486321== Profiling application: ./CUDA_720p
==486321== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   96.46%  387.48ms         8  48.435ms  47.588ms  48.716ms  find_best_match_kernel(unsigned char const *, int, int, unsigned char const *, int, int, double, double*)
                    3.43%  13.783ms        17  810.76us  17.056us  1.9211ms  [CUDA memcpy HtoD]
                    0.06%  234.98us        25  9.3990us  1.5360us  27.072us  [CUDA memcpy DtoH]
                    0.03%  112.86us        24  4.7020us  2.8160us  8.5760us  find_max_kernel(double const *, int const *, int, double*, int*, bool)
                    0.02%  71.648us         9  7.9600us  3.4560us  8.6080us  color_to_grayscale_kernel(unsigned char const *, unsigned char*, int, int, int)
      API calls:   75.66%  1.33829s        74  18.085ms  3.5090us  1.32646s  cudaMalloc
                   22.40%  396.22ms        74  5.3543ms  4.0810us  48.697ms

In [24]:
%%bash
nsys profile  -o CUDA_720p ./CUDA_720p

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



Starting CUDA 720p object tracker (NO TILLING VERSION)
Template loaded successfully (251 x 351).
Frame 0: Best match found at (x=426, y=131) with similarity: 0.999818
Frame 1: Best match found at (x=453, y=141) with similarity: 0.976094
Frame 2: Best match found at (x=493, y=132) with similarity: 0.977169
Frame 3: Best match found at (x=556, y=148) with similarity: 0.944861
Frame 4: Best match found at (x=526, y=144) with similarity: 0.974873
Frame 5: Best match found at (x=468, y=131) with similarity: 0.974679
Frame 6: Best match found at (x=401, y=139) with similarity: 0.964025
Frame 7: Best match found at (x=389, y=143) with similarity: 0.970891
Generating '/tmp/nsys-report-e46d.qdstrm'
[1/1] [========================100%] CUDA_720p.nsys-rep
Generated:
	/home/jupyter-dani_ughoc@dlsu.ed-85ec4/CUDA_720p.nsys-rep


# CUDA 720p With Tilling

In [25]:
%%writefile CUDA_720p_tiled.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

#define BLOCK_SIZE 16
#define REDUCTION_THREADS 256
#define TILE_DIM 32 

__global__ 
void color_to_grayscale_kernel(const unsigned char *color_img, unsigned char *gray_img, int width, int height, int channels) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int gray_idx = y * width + x;
        int color_idx = gray_idx * channels; 

        unsigned char r = color_img[color_idx];
        unsigned char g = color_img[color_idx + 1];
        unsigned char b = color_img[color_idx + 2];

        gray_img[gray_idx] = (unsigned char)(0.299f * r + 0.587f * g + 0.114f * b);
    }
}

__global__ 
void find_best_match_tiled_kernel(const unsigned char *frame_gray, int frame_w, int frame_h,
                                  const unsigned char *template_gray, int template_w, int template_h,
                                  double template_norm, double *similarity_map) {

    __shared__ unsigned char s_template_tile[TILE_DIM * TILE_DIM];

    int x = blockIdx.x * blockDim.x + threadIdx.x; 
    int y = blockIdx.y * blockDim.y + threadIdx.y; 

    int tid = threadIdx.y * blockDim.x + threadIdx.x;
    int num_threads = blockDim.x * blockDim.y;

    double dot_product = 0.0;
    double patch_norm_sq = 0.0;

    bool active = (x <= frame_w - template_w && y <= frame_h - template_h);

    for (int ty_start = 0; ty_start < template_h; ty_start += TILE_DIM) {
        for (int tx_start = 0; tx_start < template_w; tx_start += TILE_DIM) {
            
            int tile_pixels = TILE_DIM * TILE_DIM;
            
            for (int i = tid; i < tile_pixels; i += num_threads) {
                int t_local_y = i / TILE_DIM;
                int t_local_x = i % TILE_DIM;
                
                int t_global_y = ty_start + t_local_y;
                int t_global_x = tx_start + t_local_x;

                if (t_global_y < template_h && t_global_x < template_w) {
                    s_template_tile[i] = template_gray[t_global_y * template_w + t_global_x];
                } else {
                    s_template_tile[i] = 0; 
                }
            }

            __syncthreads();

            if (active) {
                int loop_h;
                if (ty_start + TILE_DIM > template_h) {
                    loop_h = template_h - ty_start;
                } else {
                    loop_h = TILE_DIM;
                }

                int loop_w;
                if (tx_start + TILE_DIM > template_w) {
                    loop_w = template_w - tx_start;
                } else {
                    loop_w = TILE_DIM;
                }



                for (int ty = 0; ty < loop_h; ++ty) {
                    for (int tx = 0; tx < loop_w; ++tx) {
                        unsigned char val_t = s_template_tile[ty * TILE_DIM + tx];
                        
                        int frame_idx = (y + ty_start + ty) * frame_w + (x + tx_start + tx);
                        unsigned char val_f = frame_gray[frame_idx];

                        dot_product += (double)val_f * val_t;
                        patch_norm_sq += (double)val_f * val_f;
                    }
                }
            }

            __syncthreads();
        }
    }

    if (active) {
        double patch_norm = sqrt(patch_norm_sq);
        double similarity = (patch_norm > 0 && template_norm > 0) ? (double)(dot_product / (patch_norm * template_norm)) : 0.0f;
        similarity_map[y * (frame_w - template_w + 1) + x] = similarity;
    }
}

__global__
void find_max_kernel(const double *similarity_map, const int *input_idxs, int map_size, 
                     double *block_max_vals, int *block_max_idxs, bool first_pass) {
    __shared__ double shared_vals[REDUCTION_THREADS];
    __shared__ int shared_idxs[REDUCTION_THREADS];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    double my_val = -2.0f;
    int my_idx = 0;

    if (idx < map_size) {
        my_val = similarity_map[idx];
        my_idx = first_pass ? idx : input_idxs[idx];
    }

    shared_vals[tid] = my_val;
    shared_idxs[tid] = my_idx;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            double diff = shared_vals[tid + s] - shared_vals[tid];
            if (diff > 1e-9f || (fabsf(diff) < 1e-9f && shared_idxs[tid + s] < shared_idxs[tid])) {
                shared_vals[tid] = shared_vals[tid + s];
                shared_idxs[tid] = shared_idxs[tid + s];
            }
        }
        __syncthreads();
    }

    if (tid == 0) {
        block_max_vals[blockIdx.x] = shared_vals[0];
        block_max_idxs[blockIdx.x] = shared_idxs[0];
    }
}

void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h);
void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y);

int main() {
    clock_t start, end;
    double total_time = 0.0;

    printf("Starting CUDA object tracker (TILED SHARED MEMORY)...\n");

    int template_w, template_h, template_c;
    unsigned char *h_template_color = stbi_load("data/template_720.jpg", &template_w, &template_h, &template_c, 3);
    if (h_template_color == NULL) {
        fprintf(stderr, "Error loading template.\n");
        return 1;
    }
    template_c = 3; 

    // GPU Allocations
    unsigned char *d_template_color, *d_template_gray;
    gpuErrchk( cudaMalloc((void**)&d_template_color, template_w * template_h * 3) );
    gpuErrchk( cudaMalloc((void**)&d_template_gray, template_w * template_h) );
    gpuErrchk( cudaMemcpy(d_template_color, h_template_color, template_w * template_h * 3, cudaMemcpyHostToDevice) );
    
    // Process Template to Grayscale
    dim3 grid_template((template_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (template_h + BLOCK_SIZE - 1) / BLOCK_SIZE);
    dim3 block_template(BLOCK_SIZE, BLOCK_SIZE);
    color_to_grayscale_kernel<<<grid_template, block_template>>>(d_template_color, d_template_gray, template_w, template_h, 3);
    
    // CPU Helper
    unsigned char *h_template_gray = (unsigned char *)malloc(template_w * template_h);
    gpuErrchk( cudaMemcpy(h_template_gray, d_template_gray, template_w * template_h, cudaMemcpyDeviceToHost) );

    double template_norm_sq = 0.0;
    for (int i = 0; i < template_w * template_h; ++i) template_norm_sq += (double)h_template_gray[i] * h_template_gray[i];
    double template_norm = sqrt(template_norm_sq);

    gpuErrchk( cudaFree(d_template_color) );
    stbi_image_free(h_template_color);

    for (int i = 0; i < 8; ++i) {
        char frame_path[128];
        sprintf(frame_path, "data/images720p/%d.jpg", i);

        int frame_w, frame_h, frame_c;
        unsigned char *h_frame_color = stbi_load(frame_path, &frame_w, &frame_h, &frame_c, 3);
        if (!h_frame_color) continue;
        frame_c = 3;

        unsigned char *d_frame_color, *d_frame_gray;
        double *d_similarity_map;
        int map_w = frame_w - template_w + 1;
        int map_h = frame_h - template_h + 1;

        gpuErrchk( cudaMalloc((void**)&d_frame_color, frame_w * frame_h * frame_c) );
        gpuErrchk( cudaMalloc((void**)&d_frame_gray, frame_w * frame_h) );
        gpuErrchk( cudaMalloc((void**)&d_similarity_map, map_w * map_h * sizeof(double)) );
        gpuErrchk( cudaMemcpy(d_frame_color, h_frame_color, frame_w * frame_h * frame_c, cudaMemcpyHostToDevice) );

        start = clock();

        dim3 grid_dim_gray( (frame_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (frame_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_gray(BLOCK_SIZE, BLOCK_SIZE);
        color_to_grayscale_kernel<<<grid_dim_gray, block_dim_gray>>>(d_frame_color, d_frame_gray, frame_w, frame_h, frame_c);
        gpuErrchk( cudaPeekAtLastError() );

        dim3 grid_dim_match( (map_w + BLOCK_SIZE - 1) / BLOCK_SIZE, (map_h + BLOCK_SIZE - 1) / BLOCK_SIZE );
        dim3 block_dim_match(BLOCK_SIZE, BLOCK_SIZE);
        
        find_best_match_tiled_kernel<<<grid_dim_match, block_dim_match>>>(
            d_frame_gray, frame_w, frame_h, d_template_gray, template_w, template_h, template_norm, d_similarity_map);
        gpuErrchk( cudaPeekAtLastError() );

        int map_size = map_w * map_h;
        int threads = REDUCTION_THREADS;
        int blocks = (map_size + threads - 1) / threads;

        double *d_block_max; int *d_block_idx;
        gpuErrchk( cudaMalloc((void**)&d_block_max, blocks * sizeof(double)) );
        gpuErrchk( cudaMalloc((void**)&d_block_idx, blocks * sizeof(int)) );

        find_max_kernel<<<blocks, threads>>>(d_similarity_map, NULL, map_size, d_block_max, d_block_idx, true);

        double *d_curr_vals = d_block_max;
        int *d_curr_idxs = d_block_idx;
        int curr_size = blocks;

        while (curr_size > 1) {
            int next_blocks = (curr_size + threads - 1) / threads;
            double *d_next_vals; int *d_next_idxs;
            gpuErrchk( cudaMalloc((void**)&d_next_vals, next_blocks * sizeof(double)) );
            gpuErrchk( cudaMalloc((void**)&d_next_idxs, next_blocks * sizeof(int)) );

            find_max_kernel<<<next_blocks, threads>>>(d_curr_vals, d_curr_idxs, curr_size, d_next_vals, d_next_idxs, false);

            if (d_curr_vals != d_block_max) { cudaFree(d_curr_vals); cudaFree(d_curr_idxs); }
            d_curr_vals = d_next_vals; d_curr_idxs = d_next_idxs;
            curr_size = next_blocks;
        }

        int global_idx;
        double max_similarity;
        gpuErrchk( cudaMemcpy(&max_similarity, d_curr_vals, sizeof(double), cudaMemcpyDeviceToHost) );
        gpuErrchk( cudaMemcpy(&global_idx, d_curr_idxs, sizeof(int), cudaMemcpyDeviceToHost) );

        gpuErrchk( cudaFree(d_block_max) ); gpuErrchk( cudaFree(d_block_idx) );
        if (d_curr_vals != d_block_max) { cudaFree(d_curr_vals); cudaFree(d_curr_idxs); }

        int best_x = global_idx % map_w;
        int best_y = global_idx / map_w;

        end = clock();
        total_time += ((double)(end - start)) * 1000.0 / CLOCKS_PER_SEC;

        printf("Frame %d: Best match (x=%d, y=%d), Sim: %f\n", i, best_x, best_y, max_similarity);

        draw_rectangle(h_frame_color, frame_w, frame_h, frame_c, best_x, best_y, template_w, template_h);
        char output_path[128];
        sprintf(output_path, "results/CUDA_720p_tiled/frame_%02d.png", i);
        stbi_write_png(output_path, frame_w, frame_h, frame_c, h_frame_color, frame_w * frame_c);

        update_template(d_frame_gray, frame_w, h_template_gray, template_w, template_h, best_x, best_y);
        gpuErrchk( cudaMemcpy(d_template_gray, h_template_gray, template_w * template_h, cudaMemcpyHostToDevice) );

        template_norm_sq = 0.0;
        for (int j = 0; j < template_w * template_h; ++j) template_norm_sq += (double)h_template_gray[j] * h_template_gray[j];
        template_norm = sqrt(template_norm_sq);

        stbi_image_free(h_frame_color);
        gpuErrchk( cudaFree(d_frame_color) );
        gpuErrchk( cudaFree(d_frame_gray) );
        gpuErrchk( cudaFree(d_similarity_map) );
    }

    gpuErrchk( cudaFree(d_template_gray) );
    free(h_template_gray);

    printf("Total GPU processing time: %f ms\n", total_time);
    return 0;
}

void draw_rectangle(unsigned char *color_img, int frame_w, int frame_h, int channels, int best_x, int best_y, int template_w, int template_h) {
    unsigned char black[] = {0, 0, 0};
    for (int x = best_x; x < best_x + template_w; ++x) {
        if (x >= 0 && x < frame_w) {
            if (best_y >= 0 && best_y < frame_h) memcpy(&color_img[(best_y * frame_w + x) * channels], black, channels);
            int bottom_y = best_y + template_h - 1;
            if (bottom_y >= 0 && bottom_y < frame_h) memcpy(&color_img[(bottom_y * frame_w + x) * channels], black, channels);
        }
    }
    for (int y = best_y; y < best_y + template_h; ++y) {
        if (y >= 0 && y < frame_h) {
            if (best_x >= 0 && best_x < frame_w) memcpy(&color_img[(y * frame_w + best_x) * channels], black, channels);
            int right_x = best_x + template_w - 1;
            if (right_x >= 0 && right_x < frame_w) memcpy(&color_img[(y * frame_w + right_x) * channels], black, channels);
        }
    }
}

void update_template(const unsigned char *d_frame_gray, int frame_w, unsigned char *h_template_gray, int template_w, int template_h, int best_x, int best_y) {
    cudaMemcpy2D(h_template_gray, template_w, 
                 d_frame_gray + best_y * frame_w + best_x, frame_w,
                 template_w, template_h, cudaMemcpyDeviceToHost);
}

Writing CUDA_720p_tiled.cu


In [26]:
%%bash
nvcc CUDA_720p_tiled.cu -o CUDA_720p_tiled -Wno-deprecated-gpu-targets "-diag-suppress=550"

In [27]:
%%bash
nvprof ./CUDA_720p_tiled

==486473== NVPROF is profiling process 486473, command: ./CUDA_720p_tiled


Starting CUDA object tracker (TILED SHARED MEMORY)...
Frame 0: Best match (x=426, y=131), Sim: 0.999818
Frame 1: Best match (x=453, y=141), Sim: 0.976094
Frame 2: Best match (x=493, y=132), Sim: 0.977169
Frame 3: Best match (x=556, y=148), Sim: 0.944861
Frame 4: Best match (x=526, y=144), Sim: 0.974873
Frame 5: Best match (x=468, y=131), Sim: 0.974679
Frame 6: Best match (x=401, y=139), Sim: 0.964025
Frame 7: Best match (x=389, y=143), Sim: 0.970891
Total GPU processing time: 399.609000 ms


==486473== Profiling application: ./CUDA_720p_tiled
==486473== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   96.54%  392.29ms         8  49.036ms  47.688ms  50.979ms  find_best_match_tiled_kernel(unsigned char const *, int, int, unsigned char const *, int, int, double, double*)
                    3.36%  13.663ms        17  803.70us  17.344us  1.9346ms  [CUDA memcpy HtoD]
                    0.05%  198.72us        25  7.9480us  1.5040us  21.152us  [CUDA memcpy DtoH]
                    0.03%  138.53us        24  5.7710us  3.5200us  10.624us  find_max_kernel(double const *, int const *, int, double*, int*, bool)
                    0.02%  73.344us         9  8.1490us  3.3280us  9.0880us  color_to_grayscale_kernel(unsigned char const *, unsigned char*, int, int, int)
      API calls:   60.62%  896.92ms        74  12.121ms  4.5750us  886.86ms  cudaMalloc
                   27.13%  401.37ms        74  5.4239ms  3.9680

In [28]:
%%bash
nsys profile  -o CUDA_720p_tiled ./CUDA_720p_tiled

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



Starting CUDA object tracker (TILED SHARED MEMORY)...
Frame 0: Best match (x=426, y=131), Sim: 0.999818
Frame 1: Best match (x=453, y=141), Sim: 0.976094
Frame 2: Best match (x=493, y=132), Sim: 0.977169
Frame 3: Best match (x=556, y=148), Sim: 0.944861
Frame 4: Best match (x=526, y=144), Sim: 0.974873
Frame 5: Best match (x=468, y=131), Sim: 0.974679
Frame 6: Best match (x=401, y=139), Sim: 0.964025
Frame 7: Best match (x=389, y=143), Sim: 0.970891
Total GPU processing time: 397.684000 ms
Generating '/tmp/nsys-report-7d59.qdstrm'
[1/1] [========================100%] CUDA_720p_tiled.nsys-rep
Generated:
	/home/jupyter-dani_ughoc@dlsu.ed-85ec4/CUDA_720p_tiled.nsys-rep
